# Variational Autoencoder for Collaborative Filtering: 
# The Trade-off between Generation and Prediction

<b><font color='grey' size=3> GROUP MEMBER: 
* Section 1 - Tonghong Chen(tc2894)
* Section 2 - Siyi Sun(ss5339), Liangliang Huang(lh2863), Yang Yang(yy2819) 


# Content

INTRODUCTION
  * Background and Motivation
  * Project Goal
  * Quick Summary

PART I: VAE for recommendation systems

- Data Re-Engineering
  * Dataset Overview
  * Challenge
  * Solution
  * Full Data Engineering Process

- Models
  * Training Trick
  * Predictive VAE
  * Inferential VAE

- Metrics and Measurement
 * Recall
 * Normalized Truncated Discounted Cumulative Gain (NDCG)
 
- Model Implementation
  * Shared Hyperparameters
  * Regularized VAE Instance
  * Inferential VAE Instance
  * Training and Validation Execution

- Prediction and Model Evaluation
  * Performance Summary

PART II: Statistical Inference on VAE's Latent Variable 
  * Clustering Using 200 Laten Variables 
  * Check Normality of Laten Variables
  * Low Dimensions Representation

CONCLUSION

REFERENCE

## Required Package

In [0]:
!pip install bottleneck
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

import os
import shutil
import sys

import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn
sn.set()

import pandas as pd

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer

import bottleneck as bn

# Introduction



## 1.1 Background and Motivation

Variational autoencoder (VAE) is a recent advancement in machine learning. It combines the traditional autoencoder with the mathematical theory of variational inference, making the model highly explicable and tunable. 

Traditionally, VAE is used as a generator to solve music and image problem, as one of its earliest application is to generate music scores. However, thanks to the advantages like weak assumptions and parallel training, VAE has been widely explored nowadays for different business cases, such as generating similar trees in a video game, or performing dimension reduction in a machine learning problem.


Excited with the potential of VAE, our project specifically focuses on the VAE-based predictive models for recommendation system, which was just brought up recently by a Netflix Reserch's paper but hasn't raise much discussion yet [1]. According to the paper, VAE-based predictive models could outshine the traditional linear-based collaborative filtering models in movie recommendation, for its capacity of dealing non-linear relationship between the user-product matrix and latent variables. 

The Netflix paper, however, takes an arguably inappropriate data preprocessing process and thus fails to justify its prediction result. Instead of ordering the user's rating history strictly by time and using only the  "past history" to predict the "most recent history", the paper randomly shuffles the user's rating history and blindly predict the hold-out. A major concern of this approach is that it tries to use the future to "predict" the past history. For example, if we know a user just watched "Iron Man II" and need to "predict" what movies he/she watched before, it's almost 100% safe to blindly guess that the user would give a high rate on "Iron Man I". On the other hand, if we only know a user has watched "Iron Man I" and need to predict what movies he/she will watch in the future, blindly guess "Iron Man II" will convincingly return a poorer result. In fact, this phenomenon does appear in our repeated experiment. We re-train the model exactly in the same way as did by the paper. The only change is that when testing the performance, we predict the responses of last few movies in time order, instead of shuffling test.
And the performance of the model decreases significantly.

So far, the Netflix Research's paper, which is arguably inconvincible due to the flaw stated in the previous paragraph, is still the only paper we can find online that discusses the possibility of VAE's applcation on recommendation system. This provides us great motivation to testify whether the VAE model for recommendation system is truly useful. 

## 1.2 Project Goal

WIth the motivation stated above in mind, we generalize the following two main goals for this project:

(1) Study the performance of VAE-based models for recommendation systems by:
  
  * Fix the misleading data processing procedure in the Netflix Research's paper and reproduce the whole work;
  
  * Introduce our own Predictive VAE model as a better choice for the collaborative filtering problem;
  
  * Introduce the Inferential VAE model as a supplement to the 3-layer Regularized VAE model stated in the Netflix Research's paper;

  * Compare VAE-based models' performance with the classic benchmark of recommendation problem, the Weighted Matrix Factorization Model (WMF). 

(2) Perform statistical inference on the latent variables, and visualize the findings with inituive explanations. 

## 1.3 Quick Summary

* The Netflix' s simple single-layer VAE model do outshine the classic state-of-art WMF in regards to two metrics, therefore it is a useful model for recommendation system

* Our Predictive VAE model performs better than the Netflix' s model in real prediction task

* The trade-off between generation and prediction make the extended VAE model a powerful tool for collaborative filtering. 


* The latent variables of VAE could serve as a nice and interesting way to do dimension reduction and visualize user clustering under low dimensions. 



# PART I: VAE for recommendation systems

According to the Netflix Research's paper, a creative and effective way to transform VAE from a generative model to a predictive model is to treat the KL-Divergence part in the loss function as a "regularization term", and applies a parameter β selected by annealing process to control the strength of regularization.

$$
L_β (x_u ; θ, ϕ) ≡ E_{q_ϕ}(z_u |x_u)[log_{pθ}(x_u |z_u)]−β · KL(q_φ(z_u |x_u)∥p(z_u))
$$

The reason behind this β is that, while the original VAE is a powerful generative model; we might not need all the statistical properties of a generative model for tackling problems in recommendation systems. In particular, if we are willing to sacrifice the ability to perform ancestral sampling, and introduce a regularization view of the Evidence Lower Bound (ELBO), we can actually achieve a trade-off between how well we can fit the data and how close the approximate posterior stays to the prior during learning.

For predictive use-case,  the β is usually set to be < 1. This means we not only are no longer optimizing a lower bound on the log marginal likelihood, but also weaken the influence of the prior constraint, making the model less able to generate novel user histories by ancestral sampling. But ultimately, our goal is to make good recommendations, not to maximize likelihood or generate imagined user histories. Treating β as a free regularization parameter therefore costs us nothing, and, as we will see, yields significant improvements in performance.

To answer how to select the optimal β, a simple anneal heuristic process is purposed by the Netflix Research's paper: "we start training with β = 0, and gradually increase β to 1. We linearly anneal the KL term slowly over a large number of gradient updates to θ, ϕ and record the best β when its performance reaches the peak. This method works well and it does not require the need for training multiple models with different values of β, which can be time-consuming." 

After several rounds of tuning, we concluded that the key methodologies purposed in the Netflix Research's paper, the regularization view of ELBO and the anneal process to select the optimal β, did contribute significantly to improve the prediction accuracy. Therefore, we adopted this regularization view and tunning technique in all the models we built.

However, as we explained in the "Background and Motivation" section, the Netflix Research's paper has a significant flaw in its data processing procedure, which makes its result less convincible. Therefore, the frist step we take is to correct this flaw and produce the right dataset for train, validation, and test. 

Then, we conetinue to produce our own models, which we named as "Predictive VAE" and "Inferential VAE".


## 2. Data Re-Engineering

### 2.1 Dataset Overview



The dataset we use in this project is "MovieLens 20M", which is a stable benchmark dataset containing 20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users. The dataset contains four columns: “userId”, “movieId”, “rating” and “timestamp”. The dataset can be downloaded at http://files.grouplens.org/datasets/movielens/ml-20m.zip


In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip

In [0]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = 'ml-20m/'

In [0]:
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)

### 2.2 Past-Future Split for Performance Evaluation (Test)

#### 2.2.1 Challenge

Recall that in the Netflix Research Paper, instead of ordering the user's rating history strictly by time and using only the "past history" to predict the "most recent history", it randomly shuffles the user's rating history and blindly predict the hold-out. Below it's the code:

In [0]:
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

A major concern of this approach is that it tries to use the future to "predict" the past history. For example, if we know a user just watched "Iron Man II" and need to "predict" what movies he/she watched before, it's almost 100% safe to blindly guess that the user has watched "Iron Man I". On the other hand, if we only know a user has watched "Iron Man I" and need to predict what movies he/she will watch in the future, blindly guess "Iron Man II" will convincingly return a poor result.


#### 2.2.2 Solution

We rewrite the fcuntion split_train_test_proportion(), force it to order the user's rating history strictly by time, and using only the "past history", in another word, the first 1-X% of data, to predict the "most recent history", in another word, the last X% of data.

In [0]:
def split_train_test_proportion(data, test_prop=0.2):
  '''
  @Function DESC: 
    * further split the validation or test data into "fold-in" set and "hold-out(prediction)" set 
    * the hold-out set considers time effect and will be the most recent records
  '''
  
  data_grouped_by_user = data.groupby('userId')
  tr_list, te_list = list(), list()

  np.random.seed(98765)

  for i, (_, group) in enumerate(data_grouped_by_user):
      group = group.sort_values(['timestamp'],ascending=True)
      n_items_u = len(group)

      idx = np.zeros(n_items_u, dtype='bool') # for example, array([False, False, False])
      idx[-int(test_prop * n_items_u):] = True

      tr_list.append(group[np.logical_not(idx)])
      te_list.append(group[idx])

#       if i % 1000 == 0:
#           print("%d users sampled" % i)
#           sys.stdout.flush()

  data_tr = pd.concat(tr_list)  # tr_list is a list of dataframes by different userid
  data_te = pd.concat(te_list)

  return data_tr, data_te

### 2.3 Past-Future Split for Training Data


After we correct the flaw of data splitting for prediction metrics, the generic regularized VAE perform much worse.  As **6.1 Performance Sumary** show, its NDCG drops from 42% to 23%.  

So we try to involve a training trick, which is then proved to perform better than the generic one. The NDCG increases to 25.6% after applying the training trick.  

The **Training Trick** would be introduced in following **3. Predictive Models**.  
Following is the data preprocess added for training trick, creating $Past\ 80\%\ Train\ Data$, which only contains earliest 80% of the train users' positive records.



In [0]:
## add train_plays_tr
## the train_plays didn't change, which is still the full record of train_users.
train_plays_tr, _ = split_train_test_proportion(train_plays) # Changed for train trick (added)

### 2.4 Full Data Engineering Process

This part introduce the the data preprocessing code of our project and the logic behind it

#### 2.4.1 HIgh-Quality Movies and Users

Since the recommendation system aims to match great movies with valuable customers who are willing to give high ratings, the dataset used in our project was filtered by only keeping movies with at least one rating greater than 3.5 and by keeping users with at least five movie-rating records. 

In [0]:
def get_count(tp, id):
    '''
    count the number of triplets by id
    
    @param dataframe tp: the raw_data dataframe
    @parm int id: either movieId or userId
    '''
    return tp[id].value_counts()

In [0]:
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for movies which were clicked on by at least min_sc users. 

    if min_sc > 0:
        moviecount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(moviecount.index[moviecount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc movies
    # After doing this, some of the movies will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and moviecount after filtering
    usercount, moviecount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
    return tp, usercount, moviecount

In [0]:
# first, only keep the movies with at least one rating greater than 3.5
raw_data = raw_data[raw_data['rating'] > 3.5]
# second, only keep the users with at least 5 movie-rating records 
raw_data, user_activity, item_popularity = filter_triplets(raw_data, min_uc=5, min_sc=0)

We could check out the sparsity after this flitering process:

In [0]:
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

After filtering, there are 9990682 watching events from 136677 users and 20720 movies (sparsity: 0.353%)


#### 2.4.2 User-Rating Sparse Matrix

After filtering, we construct a binary click matrix in which each row represents a user’s click history and each column a distinct movie. If user(u) has rated movie(i), then we assign entry (u,i) to be 1, regardless the rating. Otherwise, we assign entry (u, i) to be 0. Splitting the binary click matrix, we obtain the train, validation, and test data set. The train data contain 80% of users and their full click history, while the validation and test data each contains the rest of 10%. 


We further split validation and test data into subsets. Before splitting, movies that didn’t exist in the train set are removed because our model is unable to predict a movie without any rating information. 

We splitting the data in two ways for later comparison. The first way, according to Liang [1], in splitting, we randomly keep 80% of their records as in-fold(train) data and 20% as held-out for users with greater than or equal to 5 ratings in both validation and test data set, and we assign the users with less than 5 ratings directly to the in-fold data. In the second way, we split the data according to the timestamp instead of random selection. Before splitting, we arrange data by timestamp, then pick the first 80% as in-fold data and the rest as held-out. We will use the fold-in data from the validation and test data set to learn user-level representations and build the metrics using the held-out data to evaluate the model performance. Processing data in this way, we obtain user-by-item matrices. 

In [0]:
np.random.seed(98765)

unique_uid = user_activity.index # [1,2,3,..., 138493] | length=136677
unique_uid = unique_uid[np.random.permutation(unique_uid.size)] # randomly shuffle the user index
# unique_uid is like [105696,  11200, 112758, ... , 136723] | length=136677

In [0]:
n_users = unique_uid.size
n_heldout_users = 10000

tr_users_index = unique_uid[:(n_users - n_heldout_users * 2)]
vd_users_index = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
te_users_inex = unique_uid[(n_users - n_heldout_users):]

#### 2.4.3 Split the train, valiation and test dataset

##### 2.4.3.1 produce the train dataset

In [0]:
train_plays = raw_data.loc[raw_data['userId'].isin(tr_users_index)]

##### 2.4.3.2 get and save the unique movieId in train dataset to disk

In [0]:
# select the unique movieId appearing in train data
unique_sid = np.unique(train_plays['movieId']) 

In [0]:
# save the list of unique movieId to disk
pro_dir = os.path.join(DATA_DIR, 'pro_sg')
if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

##### 2.4.3.3 produce the validation and test dataset

In [0]:
# make sure the all movieIds in validation set have appeared in the train dataset
vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users_index)]
vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]  
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

In [0]:
# make sure the all movieIds in test set have appeared in the train dataset
test_plays = raw_data.loc[raw_data['userId'].isin(te_users_inex)]
test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

by now, we have produced the following dataframes:
- train_plays
- vad_plays_tr, vad_plays_te
- test_plays_tr, test_plays_te

##### 2.4.3.4 Save the data into (user_index, item_index) format

In [0]:
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid)) 
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid)) 
                             
def numerize(tp):
  uid = list(map(lambda x: profile2id[x], tp['userId']))  # map the tp['userId'] to its corresponding shuffled index
  sid = list(map(lambda x: show2id[x], tp['movieId'])) # map the tp['movieId'] to its corresponding shuffled index
   # map tp to the corresponding shuffled uid and sid
  return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [0]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

In [0]:
# Changed for train trick (added)
train_data_tr = numerize(train_plays_tr)
train_data_tr.to_csv(os.path.join(pro_dir, 'train_tr.csv'), index=False)

In [0]:
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

In [0]:
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

In [0]:
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

In [0]:
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)

#### 2.4.4 construct the user-rating sparse mMatrix for training, valiation and test

In [0]:
def load_train_data(csv_file):
  tp = pd.read_csv(csv_file)
  n_users = tp['uid'].max() + 1

  rows, cols = tp['uid'], tp['sid']
  data = sparse.csr_matrix((np.ones_like(rows),
                           (rows, cols)), dtype='float64',
                           shape=(n_users, n_items))
  return data

In [0]:
def load_tr_te_data(csv_file_tr, csv_file_te):
  tp_tr = pd.read_csv(csv_file_tr)
  tp_te = pd.read_csv(csv_file_te)

  start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
  end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

  rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
  rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

  data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                           (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
  data_te = sparse.csr_matrix((np.ones_like(rows_te),
                           (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
  return data_tr, data_te

In [0]:
DATA_DIR = 'ml-20m/'
pro_dir = os.path.join(DATA_DIR, 'pro_sg')
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

In [0]:
train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))

In [0]:
# Changed for train trick (added)
train_data_tr = load_train_data(os.path.join(pro_dir, 'train_tr.csv'))

In [0]:
vad_data_tr, vad_data_te = load_tr_te_data(os.path.join(pro_dir, 'validation_tr.csv'),
                                           os.path.join(pro_dir, 'validation_te.csv'))

In [0]:
test_data_tr, test_data_te = load_tr_te_data(os.path.join(pro_dir, 'test_tr.csv'),
                                             os.path.join(pro_dir, 'test_te.csv'))

by now, we have produced the user-rating matrix for train, validation and test:
- train_data
- vad_data_tr, vad_data_te
- test_data_tr, test_data_te

## 3. Predictive Models

After fixing the data preprocessing problem, we conetinue to produce our own models, which we named as "Predictive VAE" and "Inferential VAE".

### 3.1 Training Trick


The annealing VAE structure can be conclude as 
$$Input \rightarrow Z \rightarrow Output\\
Minimize(|Output-Label|+\beta \cdot KL)$$
,where the$|Output-Label|$ is the cross-entropy loss between Output & Label. 
For the generic VAE $Input=Label$, which focus on generate(recover) the original data from sparse encoding $Z$.  
The Netflix paper's generic Model just treats $Input=Label=Train\ Data$

However, in Recommending System,  we should reconsider the problem on generate(recover) and predition. Just as we mention before, we want to use the past watching history to recommend user some movies they may want to watch & like in the future. Thus, the model need to have a "predictive" ability rather than just recovering/generative ability.  

To enhance the VAE's "predictive" ability, we apply a small training trick, which treats: 
$$Input=Past\ 80\%\ Train\ Data\\
(Unchanged)Label=Train\ Data(all\ data)$$
The data re-engineer part is included in **2.3 Past-Future Split for Training Data**


### 3.2 Predictive VAE

#### 3.2.1 Model Overview

The first model we build is a partially-regularized variational autoencoder with multinomial likelihood. The variational autoencoder contains a deep latent Gaussian model. The deep latent Gaussian model assumes that the observed data were generated by sampling latent variables from a standard Gaussian prior. 

In this model, we use $u$ and $i$ to represent the index of user and item respectively. Let $u$ be a local observed variable and $z$ its corresponding local latent variable. We start by sampling $z_u$ with K dimension from a standard Gaussian prior for each user. We set the dimension of the latent representation K to 200 and any hidden layer to 600, because this is the best set of values we found through tests and trials. 

The below graph shows how our model looks like:
![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/vae_ppt.jpg?raw=true =800x400)


$z_u$ is related to each user $u$ by being transformed to $π(z_u)$, a probability distribution over $I$ items, via a non-linear function, a multilayer perceptron with parameters $θ$, $f_θ(·)$. That is, one user’s click history $x_u$ can be drawn from a multinomial distribution given the total number of clicks $N_u=\sum_i x_{ui}$ with probability $π(z_u)$. 


$$
\mathbf{z}_u \sim \mathcal{N}(0, \mathbf{I}_K),  \pi(\mathbf{z}_u) \propto \exp\{f_\theta (\mathbf{z}_u)\},\\
\mathbf{x}_u \sim \mathrm{Mult}(N_u, \pi(\mathbf{z}_u))
$$

We wish to optimize the model parameters θ with respect to the marginal likelihood $p_θ(x)$. Since the posterior distribution is intractable, we use variational inference [4, 5] to approximate $p_θ(z_u | x_u )$ with a simpler variational distribution $q_φ(z_u)$, where $q_φ(z_u)$ is defined as a fully factorized (diagonal) Gaussian distribution: $q_φ(z_u) = N(μ_u,diag(σ_u^2))$ and the local variational parameters $ϕ$ = {$μ_u,σ_u$} . The objective of the variational inference is to optimize the free parameters $ϕ$ by minimizing the Kullback Leiber divergence. Since we have two free parameters for each user, when the number of user becomes extremely large, the free parameters will be hard to compute. Therefore, a data-dependent function $q_φ(x_u)$ is used to replace individual variational parameters and the variational distribution is rewritten as $q_φ(z_u |x_u) = N(μ_φ(x_u),diag(σ_φ^2(x_u))$. 

Given the inference model, we can approximate the intractable posterior $p_\theta(z_u| x_u)$ with parameters of the optimized variational distribution $q_φ (z_u | x_u )$ using input $x_u$. In this model, $q_\phi$ is the encoder and $p_\theta$ is the decoder. 

#### 3.2.2 The Evidence Lower Bound (ELBO)
We then lower-bound the log marginal likelihood of the data, according to the standard of learning latent-variable models. Below is the objective function, which is commonly known as the evidence lower bound (ELBO):

$$
logp(x_u;θ)≥E_{qφ}(z_u |x_u)[log_{pθ}(x_u |z_u)]−KL(q_φ(z_u |x_u)∥p(z_u)) ≡ L(x_u;θ,φ)
$$

Noticing that maximizing ELBO with respect to the model parameter $θ$ approximately maximizes the log marginal likelihood, while maximizing it with respect to the variational parameter $ϕ$ minimizes the KL term. Also, the KL term determines the tightness of the lower bound, where if KL divergence is zero, then $q_ϕ(z|x)=p_θ(z|x)$. Therefore, it can be treated as a regularization term. If we treat the KL term as a regularization term, it is natural to introduce a coefficient $\beta$ to control the strength of regularization [1]. We used the KL annealing [6] to select the value of $\beta\in [0,1]$, adjusting results from Higgins et al [7]. In the training process, we linearly anneal the KL term from $\beta = 0$ to $\beta =1$ over $θ , φ$ on a large number of gradient updates (200,000) to find the cap for $\beta$, because annealing is much more efficient than grid search. Then, we linearly anneal the KL term again from $\beta = 0$ to $\beta_{cap}$ to find the VAE parameter $θ , φ$. 

#### 3.2.3 The Reparametrization Trick
We also use a reparametrization trick [2, 3] to deal with the challenge of taking gradients with respect to $φ$ through sampling $z_u ∼ q_φ$ and optimizing it by stochastics gradient ascent to obtain an unbiased estimate of ELBO. The trick is sampling $ε∼N(0,I_K)$ and reparametrize $z_u =μ_φ(x_u)+ε⊙σ_φ(x_u)$.  

#### 3.2.4 Layers
In our model, we put 3 hidden layers because it outperforms other choices. We use a tanh non-linearity as the activation function between layers. But there is no activation function to the $g_φ (·)$ because we use its output as mean and variance of a Gaussian random variable. the dropout probability at the input layer is 0.5 and there is no weight decay in our model. The model is trained by Adam [8] with batch size of 500 users and 200 epochs. The following section will present our implementation for a Regularized VAE model.


 #### 3.2.5 Predictive VAE by TensorFlow

In [0]:
class RegularizedVAE(object):
    '''
    p_dim: dimensions for each layer in encoders(input, layer 1,..., ouput)
    q_dim: dimensions for decoders(input, layer 1,..., output)
    lam: regularization term for KL Divergence
    lr: Learning rate
    '''
   
    
    def __init__(self, p_dims, q_dims=None, lam=0.01, lr=1e-3, random_seed=None): # p_dims = [200, 600, n_items=len(unique_sid)]
        self.p_dims = p_dims
        if q_dims is None: # if doesn't specify q_dims, make it the inverse of p_dims
            self.q_dims = p_dims[::-1] # q_dims = [n_items, 600, 200]
        else:
            assert q_dims[0] == p_dims[-1], "Input and output dimension must equal each other for autoencoders."
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q-network must match."
            self.q_dims = q_dims
        self.dims = self.q_dims + self.p_dims[1:] # dims = [n_items, 600, 200, 600, n_items]
        
        self.lam = lam
        self.lr = lr
        self.random_seed = random_seed

        self.construct_placeholders()
    
    
    def construct_placeholders(self):
        self.input_ph = tf.placeholder(
            dtype=tf.float32, shape=[None, self.dims[0]],name='Input_variable')
        self.label_ph = tf.placeholder(
            dtype=tf.float32, shape=[None, self.dims[0]],name='Lable_variable') # Changed for train trick
      
        #dropout probability
        self.keep_prob_ph = tf.placeholder_with_default(1.0, shape=None,name='dropoff_probability')

        # placeholders with default values when scoring
        self.is_training_ph = tf.placeholder_with_default(0., shape=None, name='placeholder_convertor')
        self.anneal_ph = tf.placeholder_with_default(1., shape=None, name='Anneal_KL')
       
      
    def build_graph(self):
        # Initialize paramaters values
        self._construct_weights()
        
        saver, logits, KL = self.forward_pass()
        log_softmax_var = tf.nn.log_softmax(logits)
        
        # log_softmax是一个向量吗？这里为什么可以这么算？
        neg_ll = -tf.reduce_mean(tf.reduce_sum(
            log_softmax_var * self.label_ph, # Changed for train trick (from input_ph to label_ph)
            axis=-1))
        
        # apply regularization to weights
        reg = l2_regularizer(self.lam)   
        reg_var = apply_regularization(reg, self.weights_q + self.weights_p)
        # tensorflow l2 regularization multiply 0.5 to the l2 norm
        # multiply 2 so that it is back in the same scale
        
        # two constrain: 1. anneal term for KL
        #                2. L2 norm for all params in encoder and decoder network(reg_var)
        neg_ELBO = neg_ll + self.anneal_ph * KL + 2 * reg_var
        
        train_op = tf.train.AdamOptimizer(self.lr).minimize(neg_ELBO)

        # add summary statistics
        tf.summary.scalar('negative_multi_ll', neg_ll)
        tf.summary.scalar('KL', KL)
        tf.summary.scalar('neg_ELBO_train', neg_ELBO)
        merged = tf.summary.merge_all()
        
        return saver, logits, neg_ELBO, train_op, merged
    
    
    def q_graph(self):
        '''
        encoder network
        '''
        mu_q, std_q, KL = None, None, None
        
        h = tf.nn.l2_normalize(self.input_ph, 1)
        h = tf.nn.dropout(h, self.keep_prob_ph)
        
        for i, (w, b) in enumerate(zip(self.weights_q, self.biases_q)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights_q) - 1:
                # self.weights_q, self.biases_q = [], [] activation function
                h = tf.nn.tanh(h)
            else:
                mu_q = h[:, :self.q_dims[-1]]
                logvar_q = h[:, self.q_dims[-1]:]
                print("the mu_q is: ",mu_q)
                print("the logvar_q is: ",logvar_q)
                std_q = tf.exp(0.5 * logvar_q)
                KL = tf.reduce_mean(tf.reduce_sum(0.5 * (-logvar_q + tf.exp(logvar_q) + mu_q**2 - 1), axis=1))
        print("the shape of mu_q is: ", tf.shape(mu_q))
        return mu_q, std_q, KL

      
    def p_graph(self, z):
        h = z
        
        for i, (w, b) in enumerate(zip(self.weights_p, self.biases_p)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights_p) - 1:
                h = tf.nn.tanh(h)
        return h

      
    def forward_pass(self):
        '''
        Compute each parts of loss function: logits and KL Divergence
        '''
      
        # encoder output
        mu_q, std_q, KL = self.q_graph()
       
        # reparametrization trick 
        epsilon = tf.random_normal(tf.shape(std_q))
        sampled_z = mu_q + self.is_training_ph * epsilon * std_q

        # p-network
        logits = self.p_graph(sampled_z)
        
        return tf.train.Saver(), logits, KL

      
    def _construct_weights(self):
        self.weights_q, self.biases_q = [], []
        
        for i, (d_in, d_out) in enumerate(zip(self.q_dims[:-1], self.q_dims[1:])):  # this will return: [0, (10000,600)] , [1, (600, 200)]
            
            if i == len(self.q_dims[:-1]) - 1:
                # last layer
                # we need two sets of parameters for mean and variance respectively
                d_out *= 2
            weight_key = "weight_q_{}to{}".format(i, i+1)
            bias_key = "bias_q_{}".format(i+1)
            
            self.weights_q.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                # Xavier initializer method - uniform for weights params
                # One thing to be optimized
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases_q.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                # Truncated Normal distribution for biases params
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights_q[-1])
            tf.summary.histogram(bias_key, self.biases_q[-1])
            
        self.weights_p, self.biases_p = [], []

        for i, (d_in, d_out) in enumerate(zip(self.p_dims[:-1], self.p_dims[1:])):
            weight_key = "weight_p_{}to{}".format(i, i+1)
            bias_key = "bias_p_{}".format(i+1)
            self.weights_p.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases_p.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights_p[-1])
            tf.summary.histogram(bias_key, self.biases_p[-1])

### 3.3 Inferential VAE

#### 3.3.1 Model Overview

The second model we build is a Inferential VAE model purposed in Sønderby's paper.

Similar to the Predictive VAE model, The Inferential VAE model is a partially-regularized variational autoencoder with multinomial likelihood, where each latent variable, instead of following an independent standard normal distribution, is auxiliary by a 1st-order Markov Chain. 

The most difference between Inferential VAE and Predictive VAE is that Inferential VAE introduces a hierarchical specification for the latent variables, which allows the lower layers of the latent variables to be highly correlated but still maintain the computational efficiency of fully factorized models. The model as below (from Sønderby's paper[10]):

![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/laddar_vae_formula.png?raw=True =800x500)

The way to derive the loss function for Inferential VAE is exactly the same as the one in Regualrized VAE. Besides, the reparametrization trick also applies to Inferential VAE.

#### 3.3.2 Layers

After several round of tuning, we use a 2-hidden-fully-connected layer between each "Di", and the same structure for decoding "Z" to X_predict as well. A graphical visualization can be found below:

![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/laddar%20vae.png?raw=true)


We use a tanh non-linearity as the activation function between layers. But there is no activation function to the $g_φ (·)$ because we use its output as mean and variance of a Gaussian random variable. the dropout probability at the input layer is 0.5 and there is no weight decay in our model. The model is trained by Adam [8] with batch size of 500 users and 200 epochs.  The following section will present our implementation for the Inferential VAE model.

#### 3.3.3 Inferential VAE by TensorFlow

In [0]:
class Inferential_VAE(object):
    '''
    p_dim: dimensions for each layer in encoders(input, layer 1,..., ouput)
    q_dim: dimensions for decoders(input, layer 1,..., output)
    lam: regularization term for KL Divergence
    lr: Learning rate
    '''
   
    
    def __init__(self, p_dims, q_dims=None, lam=0.01, lr=1e-3, random_seed=None): # p_dims = [200, 600, n_items=len(unique_sid)]
      self.p_dims = p_dims
      if q_dims is None: # if doesn't specify q_dims, make it the inverse of p_dims
          self.q_dims = p_dims[::-1] # q_dims = [n_items, 600, 200]
      else:
          assert q_dims[0] == p_dims[-1], "Input and output dimension must equal each other for autoencoders."
          assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q-network must match."
          self.q_dims = q_dims
      self.dims = self.q_dims + self.p_dims[1:] # dims = [n_items, 600, 200, 600, n_items]

      self.lam = lam
      self.lr = lr
      self.random_seed = random_seed
      self._initializer = tf.contrib.layers.xavier_initializer(seed=self.random_seed)

      self.construct_placeholders()
    
    
    def construct_placeholders(self):
      self.input_ph = tf.placeholder(
          dtype=tf.float32, shape=[None, self.dims[0]],name='Input_variable')

      #dropout probability
      self.keep_prob_ph = tf.placeholder_with_default(1.0, shape=None,name='dropoff_probability')

      # placeholders with default values when scoring
      self.is_training_ph = tf.placeholder_with_default(0., shape=None, name='placeholder_convertor')
      self.anneal_ph = tf.placeholder_with_default(1., shape=None, name='Anneal_KL')
       
      
    def build_graph(self):
      
      # Initialize paramaters values
      self._construct_weights()

      saver, logits, KL = self.forward_pass()
      log_softmax_var = tf.nn.log_softmax(logits)

      neg_ll = -tf.reduce_mean(tf.reduce_sum(
          log_softmax_var * self.input_ph,
          axis=-1))

      # apply regularization to weights
      reg = l2_regularizer(self.lam)   
      reg_var = apply_regularization(reg, self.weights_q + self.weights_p)
      # tensorflow l2 regularization multiply 0.5 to the l2 norm
      # multiply 2 so that it is back in the same scale

      # two constrain: 1. anneal term for KL
      #                2. L2 norm for all params in encoder and decoder network(reg_var)
      neg_ELBO = neg_ll + self.anneal_ph * KL + 2 * reg_var

      train_op = tf.train.AdamOptimizer(self.lr).minimize(neg_ELBO)

      # add summary statistics
      tf.summary.scalar('negative_multi_ll', neg_ll)
      tf.summary.scalar('KL', KL)
      tf.summary.scalar('neg_ELBO_train', neg_ELBO)
      merged = tf.summary.merge_all()

      return saver, logits, neg_ELBO, train_op, merged
    
    
    '''
    @Ladder-style q_graph
    '''
    def q_graph(self):

      mu_q, std_q, KL = None, None, None

      h = tf.nn.l2_normalize(self.input_ph, 1)
      h = tf.nn.dropout(h, self.keep_prob_ph)
      
      mu_q = []
      logvar_q = []      

      for i, (w, b) in enumerate(zip(self.weights_q, self.biases_q)):

        h = tf.matmul(h, w) + b
        h = tf.nn.tanh(h)
        
        if (i+1) % (len(self.q_dims) - 1) == 0:  # if it is the "Di" layer     
                    
          # compute mu_q_zi and var_q_zi from Di
          mu_q_zi = h   # I simply use the easiest linear function
          var_q_zi = tf.math.softplus(h) 
          logvar_q_zi = tf.math.log(var_q_zi)
          
          # insert the result into the output array
          mu_q.append(mu_q_zi)
          logvar_q.append(logvar_q_zi)
          
          # replace Di with mu_q_zi
          h = mu_q_zi
      
      # output  
      mu_q = tf.concat(mu_q, axis=1)
      logvar_q = tf.concat(logvar_q, axis=1)
      std_q = tf.exp(0.5 * logvar_q)
      KL = tf.reduce_mean(tf.reduce_sum(0.5 * (-logvar_q + tf.exp(logvar_q) + mu_q**2 - 1), axis=1))
      return mu_q, std_q, KL
          

      
    def p_graph(self, z):
      h = z

      for i, (w, b) in enumerate(zip(self.weights_p, self.biases_p)):
          h = tf.matmul(h, w) + b

          if i != len(self.weights_p) - 1:
              h = tf.nn.tanh(h)
      return h

      
    def forward_pass(self):
      '''
      Compute each parts of loss function: logits and KL Divergence
      '''

      # encoder output
      mu_q, std_q, KL = self.q_graph()

      # reparametrization trick 
      epsilon = tf.random_normal(tf.shape(std_q))
      sampled_z = mu_q + self.is_training_ph * epsilon * std_q

      # p-network
      logits = self.p_graph(sampled_z)

      return tf.train.Saver(), logits, KL

      
    def _construct_weights(self):
      
      '''
      @UPDATE: initialize the q as: X -> network -> 1 -> network -> 1 -> ... ->1 -> network
      '''
      self.weights_q, self.biases_q = [], []

      input_size = self.q_dims[0]
      latent_size = self.q_dims[-1]
      repeated_net_struct = [1] + self.q_dims[1:-1] +[1] # di -> network -> di+1


      # from X to d1
      for i, (d_in, d_out) in enumerate(zip(self.q_dims[:-1], self.q_dims[1:-1] + [1])):  
        # this will return: [0, (10000,600)] , [1, (600, 1)]
        weight_key = "weight_q_{}to{}".format(i, i+1)
        bias_key = "bias_q_{}".format(i+1)

        self.weights_q.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                # Xavier initializer method - uniform for weights params
                # One thing to be optimized
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
        self.biases_q.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                # Truncated Normal distribution for biases params
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))

      # from d1 to dK
      for i, (d_in, d_out) in enumerate(list(zip(repeated_net_struct[:-1], repeated_net_struct[1:])) * (latent_size-1)): 
        index = i + len(self.q_dims) - 1  # following the "X to d1" process

        weight_key = "weight_q_{}to{}".format(index, index+1)
        bias_key = "bias_q_{}".format(index+1)

        self.weights_q.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                # Xavier initializer method - uniform for weights params
                # One thing to be optimized
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
        self.biases_q.append(tf.get_variable(
            name=bias_key, shape=[d_out],
            # Truncated Normal distribution for biases params
            initializer=tf.truncated_normal_initializer(
                stddev=0.001, seed=self.random_seed)))


      ############################################################

      self.weights_p, self.biases_p = [], []

      for i, (d_in, d_out) in enumerate(zip(self.p_dims[:-1], self.p_dims[1:])):
          weight_key = "weight_p_{}to{}".format(i, i+1)
          bias_key = "bias_p_{}".format(i+1)
          self.weights_p.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
          self.biases_p.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
          
          # add summary stats
          tf.summary.histogram(weight_key, self.weights_p[-1])
          tf.summary.histogram(bias_key, self.biases_p[-1])

## 4. Metrics and Measurement

To measure the performance of our models, we use the same two metrics inherited from the Netlif Research's paper.

### 4.1 Recall Metrics
The Recall metrics compare the predicted rank of the hold-out items with their true rank for each user. 

$R(u, w):= \frac{\sum_{r=1}^{R} I(w(r) \in I_u)}{min(R, I_u)}$


R is the rank, standing for items ranked in first R. Recall metrics considers all the items ranked in first R equally important. $w(r)$ is the item at rank r. $I[·]$ is an indicator function and $I_u$ is the set of held-out items that user u clicked on. 

In [0]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)  # get the index of X_pred, where the first k numbers are the largest k numbers over all
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True  # set the selected k number to be True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

### 4.2 Normalized Truncated Discounted Cumulative Gain (NDCG) Metrics
NDCG is the DCG linearly normalized to [0, 1] after dividing by the best possible DCG@R, where all the held-out items are ranked at the top. DCG is shown as follows:

$R(u,w) := \sum_{r=1}^{R} \frac{2^{I(w(r) \in I_u)} - 1}{log(r+1)}$

Different from Recall, NDCG metrics evaluates higher ranks more than lower ranks.

In [0]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

## 5. Model Implementation

in this module, we presents the implementation with optimal hyperparameters of our models.

### 5.1 Shared Hyperparameters

In [0]:
N = train_data.shape[0]
idxlist = np.arange(N)

# training batch size
batch_size = 500
batches_per_epoch = int(np.ceil(float(N) / batch_size))

N_vad = vad_data_tr.shape[0]
idxlist_vad = np.arange(N_vad)

# validation batch size (since the entire validation set might not fit into GPU memory)
batch_size_vad = 2000

In [0]:
ndcg_var = tf.Variable(0.0, name='ndcg_scalar')
ndcg_dist_var = tf.placeholder(dtype=tf.float64, shape=None, name='ndcg_histogram')
ndcg_summary = tf.summary.scalar('ndcg_at_k_validation', ndcg_var)
ndcg_dist_summary = tf.summary.histogram('ndcg_at_k_hist_validation', ndcg_dist_var)

rc_var = tf.Variable(0.0, name='rc_scalar')
rc_dist_var = tf.placeholder(dtype=tf.float64, shape=None, name='rc_histogram')
rc_summary = tf.summary.scalar('rc_at_k_validation', rc_var)
rc_dist_summary = tf.summary.histogram('ir_at_k_hist_validation', rc_dist_var)

merged_valid = tf.summary.merge([ndcg_summary, ndcg_dist_summary, rc_summary, rc_dist_summary])

### 5.2 Predictive VAE Instance

For the predictive VAE model, we set both the generative function $f_\theta(\cdot)$ and the inference model $g_\phi(\cdot)$ to be 3-layer multilayer perceptron (MLP) with symmetrical architecture. 

The generative function is a [200 -> 600 -> n_items] MLP, which means the inference function is a [n_items -> 600 -> 200] MLP. Thus the overall architecture for the Multi-VAE^{PR} is [n_items -> 600 -> 200 -> 600 -> n_items].

In [0]:
# dimesnions of the decoder
p_dims = [200, 600, n_items]

n_epochs = 100  # the default n_epochs = 200

# the total number of gradient updates for annealing
total_anneal_steps = 200000

# largest annealing parameter
anneal_cap = 0.2

In [0]:
tf.reset_default_graph()
vae = RegularizedVAE(p_dims, lam=0.0, random_seed=98765)
saver, logits_var, loss_var, train_op_var, merged_var = vae.build_graph()

Set up logging and checkpoint directory

- Change all the logging directory and checkpoint directory to somewhere of your choice
- Monitor training progress using tensorflow by: `tensorboard --logdir=$log_dir`v

### 5.3 Inferential VAE Instance

For the regularized VAE model, we set both the generative function $f_\theta(\cdot)$ and the inference model $g_\phi(\cdot)$ to be 3-layer multilayer perceptron (MLP) with symmetrical architecture. 

The generative function is a [200 -> 600 -> n_items] MLP, which means the inference function is a [n_items -> 600 -> 200] MLP. Thus the overall architecture for the Multi-VAE^{PR} is [n_items -> 600 -> 200 -> 600 -> n_items].

In [0]:
# dimesnions of the decoder
p_dims = [100, 300, 300, n_items]
q_dims = [n_items, 300, 300, 100]

n_epochs = 100  # the default n_epochs = 200

# the total number of gradient updates for annealing
total_anneal_steps = 15000 

# largest annealing parameter
anneal_cap = 1.

In [0]:
tf.reset_default_graph()
vae = Inferential_VAE(p_dims, lam=0.0, random_seed=98765)
saver, logits_var, loss_var, train_op_var, merged_var = vae.build_graph()

Set up logging and checkpoint directory

- Change all the logging directory and checkpoint directory to somewhere of your choice
- Monitor training progress using tensorflow by: `tensorboard --logdir=$log_dir`v

### 5.4 Training and Validation Execution

Below is the executor code. We design this executor to be compatible with both Predictive VAE and Inferential VAE

In [0]:
ndcgs_vad = []
rc_vad = []

with tf.Session() as sess:

    init = tf.global_variables_initializer()
    sess.run(init)

    best_ndcg = -np.inf

    update_count = 0.0
    

    for epoch in range(n_epochs):  
        # real-time info print-out
        print("{} epoches have been completed".format(epoch))
          
        np.random.shuffle(idxlist)  # idxlist = range(N) where N = train_data.shape[0]
        # train for one epoch
        for bnum, st_idx in enumerate(range(0, N, batch_size)):  # batch_size=500
            end_idx = min(st_idx + batch_size, N)
#             X = train_data[idxlist[st_idx:end_idx]]
            X,Y = train_data_tr[idxlist[st_idx:end_idx]],train_data[idxlist[st_idx:end_idx]] # Changed for train trick
            
            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')
            
            # Changed for train trick (Added)
            if sparse.isspmatrix(Y):
                Y = Y.toarray()
            Y = Y.astype('float32')      
            
            if total_anneal_steps > 0:
                anneal = min(anneal_cap, 3. * update_count / total_anneal_steps) # Changed anneal rates. Previous speed is 1.
            else:
                anneal = anneal_cap
            
            feed_dict = {vae.input_ph: X, 
                         vae.label_ph: Y, # Changed for train trick(Added)
                         vae.keep_prob_ph: 0.5, 
                         vae.anneal_ph: anneal,
                         vae.is_training_ph: 1}        
            sess.run(train_op_var, feed_dict=feed_dict)
            
            # write the summary to the log
            if bnum % 100 == 0:
                summary_train = sess.run(merged_var, feed_dict=feed_dict)
                summary_writer.add_summary(summary_train, 
                                           global_step=epoch * batches_per_epoch + bnum) 
            
        '''
        @Validation Process
        '''
        # compute validation NDCG
        ndcg_dist = []
        rc_rate_dist = []
        
        for bnum, st_idx in enumerate(range(0, N_vad, batch_size_vad)):  # N_vad=10000 | batch_size_vad = 2000
            end_idx = min(st_idx + batch_size_vad, N_vad)
            X = vad_data_tr[idxlist_vad[st_idx:end_idx]]

            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')
        
            pred_val = sess.run(logits_var, feed_dict={vae.input_ph: X} )
            # exclude examples from training and validation (if any)
            pred_val[X.nonzero()] = -np.inf  # this will make the NDCG smaller

            ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, vad_data_te[idxlist_vad[st_idx:end_idx]], k=100))
            rc_rate_dist.append(Recall_at_k_batch(pred_val, vad_data_te[idxlist_vad[st_idx:end_idx]], k=100))
        
        ndcg_dist = np.concatenate(ndcg_dist)
        ndcg_ = ndcg_dist.mean()
        ndcgs_vad.append(ndcg_)
        
        rc_dist =  np.concatenate(rc_rate_dist)
        rc_ = rc_dist.mean()
        rc_vad.append(rc_)
        
        merged_valid_val = sess.run(merged_valid, feed_dict={ndcg_var: ndcg_, ndcg_dist_var: ndcg_dist, 
                                                             rc_var: rc_, rc_dist_var: rc_dist})
        summary_writer.add_summary(merged_valid_val, epoch)

        # update the best model (if necessary) and save the best model to the chkpt_dir folder
        if ndcg_ > best_ndcg:
            saver.save(sess, '{}/model'.format(chkpt_dir))
            best_ndcg = ndcg_

## 6. Prediction and Model Evaluation

Now, since we have both a generative trained model (decoder)  $f_θ (·)$ and an inference model (encoder) $g_φ(·)$, we can use them to make a prediction. With a user's click history x, we rank all the items based on $f_θ (z)$, a un-normalized predicted multinomial probability, where Z is the mean of variational distribution $μ_φ (x)$. The predicted rank is given by sorting the unnormalized multinomial probability $f_θ (z)$.

Let's reflect the two metrics we define earlier:

__* Recall__

  The Recall metrics compare the predicted rank of the hold-out items with their true rank for each user. 

  $R(u, w):= \frac{\sum_{r=1}^{R} I(w(r) \in I_u)}{min(R, I_u)}$


  R is the rank, standing for items ranked in first R. Recall metrics considers all the items ranked in first R equally important. $w(r)$ is the item at rank r. $I[·]$ is an indicator function and $I_u$ is the set of held-out items that user u clicked on. 

__* Normalized Truncated Discounted Cumulative Gain (NDCG)__

NDCG is the DCG linearly normalized to [0, 1] after dividing by the best possible DCG@R, where all the held-out items are ranked at the top. DCG is shown as follows:

$R(u,w) := \sum_{r=1}^{R} \frac{2^{I(w(r) \in I_u)} - 1}{log(r+1)}$

Different from Recall, NDCG metrics evaluates higher ranks more than lower ranks.




### 6.1 Performance Sumary 

#### 6.1.1 Predictive VAE

We use the benchmark Weighted Matrix Factorization Model (WMF) [9] to compare the our result and that of Liang’s paper [1]. The evaluation metrics we used is Recall@20. That is Recall at rank = 20. In the paper, the Recall = 0.3608, while splitting data according to timestamp, our Recall = 0.1816. It is reasonable to obtain a lower Recall after splitting data according to timestamp because some of the easier predictions are eliminated, i.e. predicting Ironman I from rating record of Ironman II. 

The following graph shows the hyperparameter tuning results of the WMF. The hyperparameter space is 
alpha : {1, 2, 5, 10, 20, 30}, factors : {32, 64, 96, 128}, reg : {0.01, 0.02, 0.03, 0.04}. 


Alpha |  Factors | Regularization
--- | --- | --- 
{1, 2, 5, 10, 20, 30} | {32, 64, 96, 128} | {0.01, 0.02, 0.03, 0.04}

##### Paper WMF
![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/paper%20wmf.jpg?raw=true = 600*200)
​
##### Our WMF
![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/our%20wmf.jpg?raw=true = 600*200)

In the graphs above, the x-axis is regularization, y-axis shows factors, and each stack represents a different alpha. We find the set of tuning parameters giving the best results in color purple, which is listed below. 

 | The set of parameters| Results
--- | --- | ---
Paper |  alpha = 10, factor = 96, reg = 0.03 | 0.3608
Ours | alpha = 5, factor = 64, reg = 0.02 | 0.1816

Since our benchmark is substantially lower than that of the paper, it is not a surprise that our model produce a lower Recall and NDCG than the paper does, which is shown below: 


 | Recall@20 | Recall@50 | NDCG
--- | --- | ---
Paper | 0.395 | 0.537 | 0.426
Ours | 0.219 | 0.372 |0.2558

Noticing that our result is much higher than that of the benchmark, this shows VAE is still a great model for recommendation system after our modifications on the model and data.

#### 6.1.2 Inferential VAE

although we've tried different network structure, drop-out rate, regularization rate and so on. The inferential VAE model underperforms the predictive VAE model in almost every case, though the difference isn't that much.

Recall@50 for Inferential VAE

![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/inferential_recall_100.png?raw=True =600x400)

NDCG@50 for Inferential VAE

![alt text](https://github.com/jokerkeny/VAE/blob/master/fig/inferential_ndcg_100.png?raw=True =600x400)

As a quick summary, we reach the following conclusion:

* The Netflix' s simple single-layer VAE model do outshine the classic state-of-art WMF in regards to two metrics, therefore it is a useful model for recommendation system

* Our Predictive VAE model performs better than the Netflix' s model in real prediction task

* The trade-off between generation and prediction make the extended VAE model a powerful tool for collaborative filtering.

# PART II: Statistical Inference on VAE's Latent Variables

As a Latent Gaussian Model, VAE is commonly used in computer vision to perform dimension reduction or low dimensional inference. One famous example is to use VAE with 2 latent variables to learn the hand-writing digital numbers. In the experiment, we can successfully see the manifold plot of the numbers. Hence, it is also worth doing inference on the collaborative filtering problem. Even though our model is an extended VAE model, focusing more on prediction than pattern simulation.

In this part, we try to do some inferences toward our VAE model. We want to know that under the trade-off between generation and prediction, what is the behaviors and performance of latent variables. Is there any change happened in the distributions of latent variables? Do our extend VAE still a good way for dimension reduction and unsupervised learning?


![替代文字](http://bjlkeng.github.io/images/generated_digits.png =300x300)

In [0]:
chkpt_dir = './chkpt/ml-20m/VAE_anneal{}K_cap{:1.1E}/{}'.format(
    total_anneal_steps/1000, anneal_cap, arch_str)
print("chkpt directory: %s" % chkpt_dir)

# Origin model
p_dims = [200, 600, n_items]

'''
# Low dimensions model
p_dims = [2, 200, 600, n_items]
'''

tf.reset_default_graph()
vae = MultiVAE(p_dims, lam=0.0, random_seed=98765)

saver, logits_var, loss_var, train_op_var, merged_var, sampled_z = vae.build_graph()

In [0]:
z=[]

with tf.Session() as sess:
    #saver.restore(sess, './model')
    saver.restore(sess, '{}/model'.format(chkpt_dir))
    
    for bnum, st_idx in enumerate(range(0, N, batch_size)):
        end_idx = min(st_idx + batch_size, N)
        X = train_data[idxlist[st_idx:end_idx]]
            
        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')           
        feed_dict = {vae.input_ph: X, }        
        train_z = sess.run(sampled_z, feed_dict={vae.input_ph: X})
        z.append(train_z)
z=pd.DataFrame(np.concatenate(z))

### 7.1 Check Normality of Latent Variables

One amazing idea for VAE is that, we can use some independent variables following standard normal distribution to simulate any distribution. Hence in the traditional VAE model, the empirical distribution of latent variables, marginalized by input X, should approximate standard normal distributions, the prior distributions of latent variables. After regularization on KL Divergence and a special training method, the empirical distribution of valent variables may be impacted by the trade-off. And actually, they are. 

We could see from the first plot, which is the empirical marginal distribution of a randomly chosen latent variable. It looks really like a Gaussian distribution. In fact, all empirical distributions of latent variables do not reject Kolmogorov-Smirnov test for normal distribution. It means that all latent variables may follow normal distribution. The mean parameters of variables approximate 0 and follow a normal distribution as well. However, the standard error(std) parameter displays strange performance. Most of std of latent variable falls in the interval (0, 0.2], while by setting the prior of z to be standard normal distribution, std should approximate 1. In our opinions, the variation of std parameter might indicates the trade-off between generation and prediction. By learning less pattern, including noise, latent variables might not be that discriminative to regenerate perfect matrix, but they learned enough information to make prediction. We think it is also a kind of trade-off between bias and variance, related to the overfitting problem.


In [0]:
i=int(np.random.uniform(0,200,1)[0])
print('Selected Latent Z: ',i)
sns.distplot(z[i])
plt.show()

![替代文字](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAeEAAAFYCAYAAABkj0SzAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz%0AAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4yLCBo%0AdHRwOi8vbWF0cGxvdGxpYi5vcmcvNQv5yAAAIABJREFUeJzt3XtwVOd9N/Dv2ftV0q60uoDAxjYG%0AAyYEGzs2Qdgy2MRpEzKxI3Dw8E5o0sa0NDM0HY+TBs/Er6emJDOd1lOnBPqm9jRR7cnrun1dK3EC%0AjmuwDSHGxhcE2AYhdFlJK2nv1/P+sTqLBFqtLrvnOWf3+5nxWNKuVj897O5Xz3OeiyTLsgwiIiJS%0AnUF0AURERJWKIUxERCQIQ5iIiEgQhjAREZEgDGEiIiJBGMJERESCmNT+gX5/UO0fKYzH40AgEBFd%0AhiaxbfJj20yO7ZIf2yY/rbSNz+ee9OvsCZeQyWQUXYJmsW3yY9tMju2SH9smP623DUOYiIhIEIYw%0AERGRIAxhIiIiQRjCREREgjCEiYiIBGEIExERCcIQJiIiEoQhTEREJAhDmIiISBCGMBERkSAMYSIi%0AIkFUP8CBiKZPlmU823EaRqMBX7h9IbxVNtElEVERMYSJNGxoNI7D71wCALz2TjdaPjMPf3Tntahx%0AWQVXRkTFwOFoIg3zD0cBAEsW1KDGZcVvT3Tj759/V3BVRFQsDGEiDfOPZEP4jhWNePJbn8P186pw%0Avi+IeCItuDIiKoaCw9HRaBSPPvooBgcHEY/H8cgjj+Duu+/O3d7a2orGxkYYjdkzG/ft24eGhobS%0AVUxUQQaGYwCAumobTEYDFja6ce7SKHqHIrimcfJDwolIPwqG8KFDh7BixQp885vfRHd3N77xjW9M%0ACGEA2L9/P5xOZ8mKJKpUA2M9YV+NHQDQ5HUAAEOYqEwUDOH7778/93FPTw97uUQq8g/HYJAkeKuy%0AE7Eaa7Mh3DMYFlkWERXJtGdHb9myBb29vXjmmWeuum3Pnj3o7u7GLbfcgt27d0OSpLyP4/E4YDIZ%0AZ1etDvl87K3kw7bJT2mboWAMdR47GhuqAQDLx147gXCyItuvEn/n6WLb5Kfltpl2CP/iF7/Ahx9+%0AiO9+97t46aWXckG7a9curFu3DtXV1di5cyc6OjqwadOmvI8TCETmXrVO+Hxu+P1B0WVoEtsmP6Vt%0AEsk0hkbjWLqwJtdWsizDYjbg/KWRims/PmfyY9vkp5W2yfeHQMHZ0adOnUJPTw8A4KabbkI6ncbQ%0A0FDu9s2bN6O2thYmkwktLS3o7OwsUslElW1gJDspS7keDAAGSUKj14HeoQgysiyqNCIqkoIhfPz4%0AcRw8eBAAMDAwgEgkAo/HAwAIBoPYsWMHEokEAODYsWNYvHhxCcslqhzKpKy6cSEMAI1eBxKpDIZG%0AYyLKIqIiKjgcvWXLFnzve9/DQw89hFgshh/84Ad48cUX4Xa7sXHjRrS0tKCtrQ1WqxXLli2bciia%0AiKbPP7Y8yVc9cavKptrsSoTewQjqqu1XfR8R6UfBELbZbPjRj36U9/bt27dj+/btRS2KiK5enqRo%0AUmZID0Ww4rpa1esiouLhjllEGqX0hCcbjgayPWEi0jeGMJFGDQxHYTEbUOUwT/h6g5drhYnKBUOY%0ASKP8IzHUVduvWndvNRtRW2VD7xB7wkR6xxAm0qBwLIloPHXVpCxFU60Dw6EEovGUypURUTExhIk0%0ASDnC8MrrwYrGcXtIE5F+MYSJNGggz/IkRRP3kCYqC9PetpKISu/wO91wu2x488M+AEBvIILD73Rf%0Adb++QLan/NYHfUikMgCAu1bNV69QIioK9oSJNCgUye5C575iZrSiymkBAIyEE6rVRETFxxAm0qBQ%0ANAkAcNktk95utxphNhowyhAm0jWGMJEGhSJJWM1GmE2Tv0QlSUKVy4LRcJIHORDpGEOYSGNkWUYo%0AmoIrz1C0osphRkaWEYlxmRKRXjGEiTQmHM32bl32qUPYacveHo4l1SiLiEqAIUykMcGIcj146hB2%0A2LKLGyJR9oSJ9IohTKQxyi5Ydqtxyvs5x0I6zF2ziHSLIUykMUoIW81Th/DlnjCHo4n0iiFMpDGx%0AxFgIWwr0hMdCOMyJWUS6xRAm0phYPA0AsBXoCVvNRhgNEmdHE+kYQ5hIY6bbE5YkCQ6bibOjiXSM%0AIUykMbFEtidc6JowkL0uHEukkc5kSl0WEZUAQ5hIY2LxFCQJeXfLGk9ZK8whaSJ9YggTaUw0kYLV%0AbIQkSQXvq0zOYggT6RNDmEhjYvE0bAWuByscnCFNpGsMYSINyWRkxJPpaV0PBsYPR3NyFpEeMYSJ%0ANCSRGpuUxZ4wUUVgCBNpSHwGM6MBwJE7xIEhTKRHDGEiDYklZ9YTtpoNYxt2cDiaSI8YwkQaMtOe%0AsCRJcNpMnB1NpFMMYSINiY/1hKc7OxrIDknHEmkkx64nE5F+MISJNGSmPWHg8lrhQDBekpqIqHQY%0AwkQaEp/hNWEAcIydKzw0yhAm0htToTtEo1E8+uijGBwcRDwexyOPPIK77747d/uRI0fw4x//GEaj%0AES0tLdi5c2dJCyYqZ/FEdg/o2fSEh4KxktRERKVTMIQPHTqEFStW4Jvf/Ca6u7vxjW98Y0IIP/HE%0AEzhw4AAaGhqwbds23HfffbjhhhtKWjRRuZrp7Gjg8lphDkcT6U/BEL7//vtzH/f09KChoSH3eVdX%0AF6qrq9HU1AQAWL9+PY4ePcoQJpqleCJ7eINlGoc3KJRdszgcTaQ/BUNYsWXLFvT29uKZZ57Jfc3v%0A98Pr9eY+93q96OrqKm6FRBUknszAZjFN6/AGRW44epTD0UR6M+0Q/sUvfoEPP/wQ3/3ud/HSSy/N%0A6E1iPI/HAZNp+kNteufzuUWXoFlsm6slkmnYrSa4XbZpf48syzAZDQhGU2XfpuX++80F2yY/LbdN%0AwRA+deoUamtr0dTUhJtuugnpdBpDQ0Oora1FfX09BgYGcvft6+tDfX39lI8XCETmXrVO+Hxu+P1B%0A0WVoEtvmapmMjHgiDY/bhmBoZr1ap82E/kCkrNuUz5n82Db5aaVt8v0hUPDC0/Hjx3Hw4EEAwMDA%0AACKRCDweDwCgubkZoVAIFy9eRCqVwqFDh7B27doilk1UOSLxFGQANuvMR4ocNhNC0WRuiRMR6UPB%0AnvCWLVvwve99Dw899BBisRh+8IMf4MUXX4Tb7cbGjRvx+OOPY/fu3QCyk7gWLVpU8qKJylEomt3/%0A2WaZ9lWinPEzpBu9jqLWRUSlU/DVbrPZ8KMf/Sjv7WvWrEF7e3tRiyKqRKFINoTts+gJKzOkA6Mx%0AhjCRjnDHLCKNUHrC1ln0hC9v2MFlSkR6whAm0ohgNAEAsFtnMxytrBXmMiUiPWEIE2lEOJo9jnAm%0AJygpeIgDkT4xhIk0QukJz2pilp3D0UR6xBAm0ojLE7NmHsIWkxE2i5HD0UQ6wxAm0ojLS5Rmt6Oc%0At8rG4WginWEIE2lEKJqEBMAy2xB2WxGOpRBPcMMOIr1gCBNpRCiahMVshGG2+7K7rQB4rjCRnjCE%0AiTQiFE3O6BzhK3mrsoc+cHIWkX4whIk0ICPLCEdTsJpn/5L0Kj1hTs4i0g2GMJEGROMpZGR5Vrtl%0AKTxV2RAOjLInTKQXDGEiDchtWTmnnjCHo4n0hiFMpAHKGuHZLk8CODGLSI8YwkQaEMz1hGcfwnar%0ACXaricPRRDrCECbSgHDuBKXZhzAAeKusHI4m0hGGMJEGBCNz7wkD2evC0XgK0XiqGGURUYkxhIk0%0AIBwrTghfvi7M3jCRHjCEiTSgmMPRABDg5CwiXWAIE2lAKJYdPi7GcDQADHFyFpEuMISJNEDpCVvm%0AGsJV3DWLSE8YwkQaEIomYbMYYTTM7vAGhXJNmEcaEukDQ5hIA8KxJJw285wfh4c4EOkLQ5hIA0LR%0AJFz2uYew1WyE02bicDSRTjCEiQRLptJIJDNw2md/eMN4HreNw9FEOsEQJhIsFM3OjC5GTxjITs6K%0AJdKIxLhhB5HWMYSJBFM26ijGNWFg/HVhDkkTaR1DmEgwZXmSs0g9Yc6QJtIPhjCRYMpZwkUbjnZz%0ArTCRXjCEiQQLj127ddqKMzErNxzNXbOINI8hTCRYqXrCHI4m0j6GMJFgpbomzIlZRNo3rfGvvXv3%0A4ve//z1SqRT+9E//FPfee2/uttbWVjQ2NsJozO55u2/fPjQ0NJSmWqIyVOyesMVshMtu5nA0kQ4U%0ADOE333wTZ86cQXt7OwKBAL7yla9MCGEA2L9/P5xOZ8mKJCpnxQ5hIDsk3ReIQpZlSNLc9qMmotIp%0AGMJr1qzBypUrAQBVVVWIRqNIp9O5ni8RzU04loIEwGEtzsQsIDs560J/CJF4qmjrj4mo+Aq+6o1G%0AIxwOBwDghRdeQEtLy1UBvGfPHnR3d+OWW27B7t27+Zc30QyEo0k4bCYY5niC0nie3JGGcYYwkYZN%0A+0/vV199FS+88AIOHjw44eu7du3CunXrUF1djZ07d6KjowObNm3K+zgejwMmU+X0on0+t+gSNItt%0AkxVNpFDltMLnc8Ptyi4vUv4/E+Pbc0FjFYBuZAyGsmrncvpdio1tk5+W22ZaIfz666/jmWeewU9/%0A+lO43RN/mc2bN+c+bmlpQWdn55QhHAhEZlmq/vh8bvj9QdFlaBLbJkuWZYyGE1hQb4XfH0QwFIPb%0AZUMwNPOZzc//+qPcxx9fGgEAdBz5BJ9cDAAA7lo1vzhFC8LnTH5sm/y00jb5/hAouEQpGAxi7969%0A+MlPfoKampqrbtuxYwcSiQQA4NixY1i8eHERyiWqDIlkBqm0XLQTlBQOa3YIWtmXmoi0qeAr/+WX%0AX0YgEMB3vvOd3Nduv/12LFmyBBs3bkRLSwva2tpgtVqxbNmyKXvBRDRRKWZGA8iFOk9SItK2giHc%0A1taGtra2vLdv374d27dvL2pRRJUiF8JFnjylzLQOM4SJNI07ZhEJlDvGsMg9YaPRAJvFyOFoIo1j%0ACBMJVKrhaABw2EyIxFKQZbnoj01ExcEQJhKo2Ccojee0mZHOyIgnM0V/bCIqDoYwkUCl7gkDQIRD%0A0kSaxRAmEqjYJyiN57RxhjSR1jGEiQQqZQg7bFwrTKR1DGEigUq1RAm43BPmMiUi7WIIEwkUjqVg%0AkCTYrcXfT93B4WgizWMIEwkUiibhtJtKcvIYh6OJtI8hTCRQOJYs2VGDRoMEm8XInjCRhjGEiQSR%0AZRnhaKoky5MUTpuZG3YQaRhDmEiQaDyFjCyXNIQdNtPYhh3pkv0MIpo9hjCRIKES7pal4AxpIm1j%0ACBMJUso1wgrH2GPzujCRNjGEiQRRI4SdypGGUc6QJtIihjCRIKXcN1rhsHOtMJGWMYSJBFEjhJ1c%0AK0ykaQxhIkFKeYyhwmFlT5hIyxjCRIKo0RM2GLJbYnJ2NJE2MYSJBMlNzCrRjlkKZcOODDfsINIc%0AhjCRIEGlJ+wobQg7bCZkZBnBCK8LE2kNQ5hIkGAkAYvZAKu5+Ccojaf0tAPBWEl/DhHNHEOYSJBQ%0ANAm33VLyn6McaTg0Gi/5zyKimWEIEwkgjw0Pu0s8FA2MD2H2hIm0hiFMJEA8mUYylSn59WBg/HA0%0Ae8JEWlO6BYpEdJXD73QDAEJjk6QisVTua6WS6wkzhIk0hz1hIgFiyey63VJPygKyG3ZI4HA0kRYx%0AhIkEiCey5/vaLKUP4eyGHSYORxNpEEOYSICYiiEMZIekA8E4N+wg0hiGMJEASghbVQphp82EdEbG%0AaDihys8joulhCBMJoOZwNAA4OEOaSJOmFcJ79+5FW1sbvvrVr+JXv/rVhNuOHDmCBx54AG1tbXj6%0A6adLUiRRuYkllRBWZ4GCk2uFiTSp4DvAm2++iTNnzqC9vR2BQABf+cpXcO+99+Zuf+KJJ3DgwAE0%0ANDRg27ZtuO+++3DDDTeUtGgivVN6wmrMjgYAx9hJTVymRKQtBUN4zZo1WLlyJQCgqqoK0WgU6XQa%0ARqMRXV1dqK6uRlNTEwBg/fr1OHr0KEOYqIBYIgVJAixmda4IKT3hALeuJNKUgu8ARqMRDocDAPDC%0ACy+gpaUFRmP2r3e/3w+v15u7r9frhd/vL1GpROUjlkjDajZCkiRVft7lDTs4HE2kJdO+IPXqq6/i%0AhRdewMGDB+f0Az0eB0wmdYbgtMDnc4suQbMqsW3cLhsAIJHMwGk35z7Pd79icTqtMBgkBKMpXbe7%0AnmsvNbZNflpum2mF8Ouvv45nnnkGP/3pT+F2X/5l6uvrMTAwkPu8r68P9fX1Uz5WIBCZZan64/O5%0A4fcHRZehSZXaNsFQDJmMjHgyjRq3BcHQ1T1Tt8s26dfnqsZlQf9QWLftXqnPmelg2+SnlbbJ94dA%0AweHoYDCIvXv34ic/+Qlqamom3Nbc3IxQKISLFy8ilUrh0KFDWLt2bXEqJipTcWVmtEqTshQetxWB%0AYAKZDDfsINKKgj3hl19+GYFAAN/5zndyX7v99tuxZMkSbNy4EY8//jh2794NALj//vuxaNGi0lVL%0AVAYub9Sh7vkpXrcN5+RRjIQT8Litqv5sIppcwXeBtrY2tLW15b19zZo1aG9vL2pRROVM7Y06FN6q%0AbPAOBWMMYSKN4I5ZRCpTNupQa8tKhcednezFZUpE2sEQJlJZLJE9xlD1nrBb6QkzhIm0giFMpDK1%0Ad8tSeJThaG5dSaQZDGEilal9jKHCOzYczZ4wkXYwhIlUdnlilrqzo6udFhgNEgLcNYtIMxjCRCq7%0APDFL3ZefwSDB47ZiYJghTKQVDGEilcUTaZhNBhgN6r/8mmqdGAknEI4lVf/ZRHQ1hjCRymKJlOrX%0AgxXz6rKHsVwaCAv5+UQ0EUOYSEWyLCM+doKSCPNqnQAYwkRawRAmUlEylUFGVn9mtGKeTwnhyjlI%0AhUjLGMJEKrq8b7TonnBIyM8nookYwkQqyp2gJCiE7VYTPG4rLg2yJ0ykBQxhIhWJOkFpvPl1TgSC%0AcUQ4Q5pIOIYwkYpyG3UImpgFAPPqxoak2RsmEo4hTKQiUYc3jJcLYc6QJhKOIUykItETswCGMJGW%0AMISJVCR6YhYAzKvlhh1EWsEQJlKRFnrCDpsZHrcV3QxhIuEYwkQqiifSMEiA2Sj2pTev1oFAMI5o%0APCW0DqJKxxAmUlE0noLNYoIkSULrmFfnAgBcGmRvmEgkcYsViSpMRpYRiadQW2UT8vMPv9Od+zgY%0ATQAAfnuiG1392d2z7lo1X0hdRJWMPWEilYyGE5BlwGk3iy4FNS4LAGAkFBdcCVFlYwgTqSQQzAae%0Awyp+AKraZQUADIcSgishqmwMYSKVDI3GAABOu/gQtpqNsFuN7AkTCcYQJlLJkNITtokfjgaAGpcV%0A4VgKyVRGdClEFYshTKSSwGg2hJ0aGI4GgCpn9rrwaIRD0kSiMISJVDIUzA5HOzQwHA0Abke2Rx6M%0A8DQlIlEYwkQqGQrGIQGwCzzGcLwqR7YnHAyzJ0wkCkOYSCWB0TjsNhMMBrEbdSjcDg5HE4nGECZS%0AQUaWMRyKa2J5ksLlMEMCh6OJRGIIE6lgNJxAOiNrYqMOhdEgwWk3I8ieMJEw0wrhzs5ObNiwAc89%0A99xVt7W2tuKhhx7Cww8/jIcffhh9fX1FL5JI77S0Ucd4bocZ0Xiay5SIBCn4jhCJRPDDH/4Qd9xx%0AR9777N+/H06ns6iFEZUTLW3UMV6V04KewQivCxMJUrAnbLFYsH//ftTX16tRD1FZ0tpGHQouUyIS%0Aq+Cf5SaTCSbT1Hfbs2cPuru7ccstt2D37t1THtPm8ThgMok70FxtPp9bdAmaVUltE0vJAACfxwG3%0Aq/ApStO5TzE0eF0A/IgnM7r499BDjaKwbfLTctvMeWxs165dWLduHaqrq7Fz5050dHRg06ZNee8f%0ACETm+iN1w+dzw+8Pii5Dkyqtbbr7RrMfyBkEQ7Ep7+t22Qrep1hMY2NhA8MRzf97VNpzZibYNvlp%0ApW3y/SEw59nRmzdvRm1tLUwmE1paWtDZ2TnXhyQqO4FgHJKknY06FFymRCTWnEI4GAxix44dSCSy%0AkzqOHTuGxYsXF6UwonIyNBpHjcuqmY06FFymRCRWwT/LT506haeeegrd3d0wmUzo6OhAa2srmpub%0AsXHjRrS0tKCtrQ1WqxXLli2bciiaqBIpG3Vc26jN61Juhxk9gxFE4ynYNbaEiqjcFXzFrVixAs8+%0A+2ze27dv347t27cXtSiicqJs1OGpUmey1Uwpy5T6A1Fco9E/FIjKFXfMIioxZaMOr9squJLJKcuU%0A+oejgishqjwMYaISUzbq0GoIK6cp9Q1VzsoFIq1gCBOVmLJRh1aHo5XTlPoD7AkTqY0hTFRiynC0%0AR6M9YWWZUl8FreEn0gqGMFGJaX04WlmmxJ4wkfoYwkQlpmzUUe2yiC4lL7fDjJFwAtF4SnQpRBWF%0AIUxUYspGHUaDdl9uVU5eFyYSQbvvCkRlQNmoQ6tD0QouUyISgyFMVEJDIzGkMzLqauyiS5kSlykR%0AicEQJiqhnrFQa/I6BFcyNS5TIhKDIUxUQr2D2RBurNV2CLscZkgSlykRqY0hTFRCSk+4UeM9YaNB%0AQm2VjT1hIpUxhIlKqHcwDAlAg8ZDGMjWyGVKROpiCBOVUM9QBN4qG6xmo+hSCqr3ZCePsTdMpB6G%0AMFGJROMpjIQSaNL49WBFgydbJ5cpEamHIUxUIr06uR6saBjrCXOZEpF6GMJEJaLMjNZLT5jD0UTq%0AYwgTlUjPUBiAfnrCvho7lykRqYwhTFQil9cIOwVXMj0mo4HLlIhUxhAmKpGeoQisFiNqNHx60pW4%0ATIlIXQxhohLIZGT0DUXR5HVAkiTR5UwbrwsTqcskugCicnL4nW4AQDCSQCqdgSRd/poejF+mdE2j%0AW3A1ROWPPWGiEhgJJwAA1S5tH2F4JS5TIlIXQ5ioBEbHQrjKqZ/rwQCHo4nUxhAmKoGR0FhPWGch%0AzGVKROpiCBOVgNITdjvMgiuZGS5TIlIXQ5ioBEbCCbjsZpiM+nuJcZkSkXr09w5BpHGJZBqxRFp3%0A14MVvC5MpB6GMFGR5WZG6zSEeZoSkXoYwkRFNhyMAwBq3PoMYWWv625/SHAlROWPIUxUZIGxEPa4%0A9bVGWLGoKbtJx7lLo4IrISp/0wrhzs5ObNiwAc8999xVtx05cgQPPPAA2tra8PTTTxe9QCK9UUK4%0ARmcbdRx+pxuH3+nG7zv9qHKY0dk1jN/+4WLu60RUfAVDOBKJ4Ic//CHuuOOOSW9/4okn8A//8A/4%0A+c9/jjfeeANnz54tepFEeiHLMgKhONwOfc6MVvg8diRTmdx6ZyIqjYLvEhaLBfv370d9ff1Vt3V1%0AdaG6uhpNTU0wGAxYv349jh49WpJCifQgGk8hkczodihaUV/DGdJEaih4gIPJZILJNPnd/H4/vF5v%0A7nOv14uurq4pH8/jccBkMs6wTP3y+bgJfj7l2DaxpAwAaKh1wu2yzfpx5vK9xXDt/Bocfb8Pw6FE%0ArhYt/HtpoQatYtvkp+W2Uf0UpUAFbYfn87nh9wdFl6FJ5do2l8Z+J6fViGAoNqvHcLtss/7eYjFJ%0AMixmA7r9oVwtov+9yvU5Uwxsm/y00jb5/hCY00Wr+vp6DAwM5D7v6+ubdNiaqFLofWa0QpIk+Grs%0ACEWT3DmLqITmFMLNzc0IhUK4ePEiUqkUDh06hLVr1xarNiLdGQ4lYDJKcNn1tWf0ZJTrwn5u2kFU%0AMgWHo0+dOoWnnnoK3d3dMJlM6OjoQGtrK5qbm7Fx40Y8/vjj2L17NwDg/vvvx6JFi0peNJEWpdIZ%0AjITi8FbZIEmS6HLmzDductbCBu1eUyPSs4IhvGLFCjz77LN5b1+zZg3a29uLWhSRHvUORpCR9T8U%0ArairsUGS2BMmKiX9LmQk0piLY9s81pRJCJuMBnjdNgyOxJFOZ0SXQ1SWGMJERdI1FsLl0hMGAJ/H%0AhowsY3BU7GxtonLFECYqkov9YQCAR2fbVU4lt2nHMEOYqBQYwkRFctEfgsNqgtVSPpvR+JSzhYcq%0AZ30/kZoYwkRFEIomEQjGy2ooGgCcNjPcDjP6A1FkMrLocojKDkOYqAi6y2xS1ngNXgcSqQy6+nm+%0AMFGxMYSJiuBCX/lNylI0erND0qcvBARXQlR+GMJERdB5cRgA4KsRe/BCKTR4HQCAjy4MC66EqPww%0AhInmSJZlnOkahsdtLYvtKq+kXBc+3TXM68JERcYQJpqj3qEIRiNJ3Ligpiy2q5xMg9eBaDzF68JE%0ARcYQJpqjMxdHAAA3LqgRXEnpNOaGpHldmKiYGMJEc3R67Frpjc3VgispnYaxyVkfnWcIExUTQ5ho%0Ajs5cHIbLbkZTnVN0KSXjtJlR77Gj8yKvCxMVE0OYaA4GR2IYGIlhcXM1DGV6PVixdKEH0XgaF/qD%0AokshKhsMYaI5UJYmlfP1YMXShdnf8aPzXKpEVCwMYaI5ONNVOSG8ZKEHACdnERUTQ5hoDk53DcNq%0AMWJhg0t0KSXncVvR4LHjzMVhpDM8X5ioGBjCRLM0GkmgZzCCG+ZXw2iojJfS0mvGrgv3cb0wUTFU%0AxjsHUQmc6RpbH1zGS5OutES5LswhaaKiYAgTzVJnBV0PVixZkL0ufJr7SBMVBUOYaJY+uhCA2WTA%0AdfOqRJeiGo/bigavA51dvC5MVAwMYaJZGI0k0NUfwg3zq2E2GUWXo6qbFtYgluB1YaJiMIkugEhP%0ADr/TDQD4tGcUAGCzGHNfqxRLFnpw+J1L+Oh8AIuaKmcUgKgU2BMmmoXeoQgAoLHWIbgS9V2enMXr%0AwkRzxRAmmoWewQjMJgNqq2yiS1FdjcuKRq8DnVwvTDRnHI4mmqFQNIlgJIlmnxMGQ3nvFz3e+GH3%0AKqcZvUMR/N/ffYy6muwJS3etmi+qNCLdYk+YaIZ6B7ND0U215XtqUiENY+cLK8PyRDQ7DGGiGark%0A68GKxlwIRwVXQqRvDGGiGZBlGT2DYdgsRtS4LKLLEcZuNaHaaUF/III0zxcmmjWGMNEMjIQTiMbT%0AaPQ6IJX5+cGFNNU5kErL6OOQNNGsTWti1pNPPomTJ09CkiQ89thjWLlyZe621tZWNDY2wmjMbliw%0Ab98+NDQ0lKZaIsGU68GVPBQaVl2VAAARO0lEQVStWFjvxkfnh9HVH8K8usq9Pk40FwVD+O2338b5%0A8+fR3t6Oc+fO4bHHHkN7e/uE++zfvx9OJ1+EVP6U68FNDGHUe+ywmA3o6gvhtpvqRZdDpEsFh6OP%0AHj2KDRs2AACuv/56jIyMIBTidnVUeSKxFLr9YbgdZrjsZtHlCGcwSFjgcyEST2FwJCa6HCJdKhjC%0AAwMD8Hg8uc+9Xi/8fv+E++zZswdbt27Fvn37IMucpEHl6fjpfqQzMq6fX13x14MVCxpcAIAL/fzD%0AnGg2ZrxZx5Uhu2vXLqxbtw7V1dXYuXMnOjo6sGnTprzf7/E4YKqgDe99PrfoEjRLb23z1of9AICV%0Ai31wO0o7M9rt0sdOXDfaLPifd3vQ7Q+r8u+pt+eMmtg2+Wm5bQqGcH19PQYGBnKf9/f3w+fz5T7f%0AvHlz7uOWlhZ0dnZOGcKBQOXMpPT53PD7g6LL0CS9tU3PYBgffjqUvRacySAYKt3wq9tlK+njF1tT%0ArRNd/SG8+1FvSTcw0dtzRk1sm/y00jb5/hAoOBy9du1adHR0AADef/991NfXw+XKDkEFg0Hs2LED%0AiUQCAHDs2DEsXry4WDUTacaRU70AgOvnVwuuRHsWjg1J/+HMQIF7EtGVCvaEV69ejeXLl2PLli2Q%0AJAl79uzBL3/5S7jdbmzcuBEtLS1oa2uD1WrFsmXLpuwFE+lRJiPjyKle2K2mXODQZfN9LkgScKLT%0Aj/s/d43ocoh0ZVrXhP/qr/5qwudLly7Nfbx9+3Zs3769uFURacj7nw4hEIzjrlXzYDJyf5sr2SxG%0ANHgc+PjSKALBODxuq+iSiHSD7yhEBbzxXg8AYO3KJsGVaNc1jdkRgiOnegRXQqQvDGGiKQQjCZzo%0AHEBTrQPXNVWJLkezFjVVwWI24LV3LiHDZYpE08YQJprC6+/2IJXOYP2q+VwbPAWL2Yjbb2rAwEgM%0A738yJLocIt1gCBPlkcnIOHSiGxazAZ+/uVF0OZp312fnAwAO/6FbcCVE+sEQJsrj3XODGByN4Y7l%0AjXDYuE1lIYuaqnBNgxsnzw4iEIyLLodIFxjCRHn85sRFAEDr6mbBlejHXZ+dh4ws4/WTl0SXQqQL%0AM962kqicHX4nO5Q6Gk7g/U+GUO+x49ylEZy7NCK4Mn24fVkD2n97Fq+dvIQv3nkNjAb+nU80Fb5C%0AiCZx+sIwAGDJwhrBleiLzWLCHcsbEQjGcfLsoOhyiDSPIUx0hWQqg7PdI7BZjFjYoN2N37WqdXV2%0AgtbLb57nqWpEBXA4mugK5y6NIJnK4Kbra2E0cFnSdClD+UB2P+mPL43i5785g3l12UMd7lo1X1Rp%0ARJrFnjDROLIs48NPAzBIEoei52Dl9bUAgJNnB9gbJpoCQ5honIv+MIKRJBbNc8Nu5UDRbHmrbGiu%0Ad8E/HEPvUOUcX0o0UwxhonE+GNvtadm1XsGV6J/SG373HCdoEeXDECYa82nvKPoCUTTVOngSUBHU%0AVdsw3+dE31AUfewNE02KIUw05lfHugCwF1xMSm/4rQ/6kEimBVdDpD0MYSIAQ6MxHPuwH9UuC+bV%0AOUSXUzZ8NXbcuKAGw6EE/u3VM6LLIdIchjBVvExGxv955SOkMzKWXevlaUlFtmapDx63Fb87eQlH%0A3+8VXQ6RpjCEqeK99MYnOPXxEFZc58X183lmcLEZjQasXzUPNosR//rKafQMhkWXRKQZDGGqaCfP%0ADuClNz5FXbUN3/rj5TCwF1wSVU4L/tcXliKeTOOfXjyFZIrXh4kAhjBVsP5ABPv/8wOYjAbs/MrN%0AcNl5XGEp3XZTA9avmoeL/jBeOPyx6HKINIG7EVBFUbZW7PaH8T/v9iCeTOPOFY34pHcUn/SOCq6u%0A/G1pXYzTF4bx6+NduPk6L1ZcVyu6JCKhGMJUUTKyjJNnB/HeuUEYJAm3LavHDc3VosuqCMofQLcs%0A8eG/3zyPf/qPU/jjtdfCZsm+DXFvaapEHI6mihGOJfGb4xfx3rlBuOxmfOFzC7F0oUd0WRWnttqG%0AVTf6EI2n8cZ7vUhnMqJLIhKGIUwVoWcwjCd+dhw9gxE0+5z4ozuvQW21TXRZFWv5tR401TrQ7Q+j%0A460uhGNJ0SURCcEQprJ36pNB/O9//T36AlGsWOTF3avnw2I2ii6rokmShLtXz8d186owMBLD/zty%0AHqcvBESXRaQ6XhOmsjU0GsPzh8/hrQ/6YDJK+JM/ugmJFIc+tcJkNGDtzY2oq7bh2Ef92PvzP+Ce%0AW5qx+fPXwWHjWxNVBj7TqewkU2m8/OYF/Peb55FIZXBtoxsP37cEi5qqJhw8T+JJkoSl13jgqbLi%0AD50DePX4RRz7sB9fa70Bty9r4LptKnsMYSor7YfO4OipPoyGE7BbjVh7UyOum1eF831BnO8Lii6P%0A8mjwOPDDHbfhlbcu4L+Onsf+//wA/3XkU2y4dQHuXN4Iq4WXD6g8SbIsy2r+QL+/ct4IfT53Rf2+%0AM1HstukfjqLj7Qs4dCLb0126sAafvdEHs0l/0x7cLhuCoZjoMoQJRhI4eXYQn/aMIiMDFrMBi5tr%0AsPvrtwKplOjyNInvNflppW18PvekX2dPmHSrZzCM35/24/jpflzoCwEAqp0W3LmiET6PXXB1NFtu%0AhwWfX9mE1Tf60Nk1jM6uYbz/yRD+5Mlf49YlPnz+5iYsbq5h75jKAkOYdCMSS6F7IIQPPg3g+Ol+%0AdPuzBwEYDRJWXl+LW5b4kEilYTTor/dLV3PYTFi1uA43X+fFJz1BXBqM4O0P+/H2h/0wGiQsaqrC%0Asms9WH2jDwvqXTz9inRpWiH85JNP4uTJk5AkCY899hhWrlyZu+3IkSP48Y9/DKPRiJaWFuzcubNk%0AxVJlGA0n8HHPKPqGIugfjsIfiOLSYBhDo/HcfQwGCQvqXVjY4MKCehcsZiPSGZkBXIaMRgNuaK7G%0AqiX1ONsVQLc/jL6hCM51j+Bs9wheeuNTuOxm3L6sAfPqnGjw2NHodcDjtjKYSfMKhvDbb7+N8+fP%0Ao729HefOncNjjz2G9vb23O1PPPEEDhw4gIaGBmzbtg333XcfbrjhhpIWTfqUkWUMB+MIBOM4PxDB%0Apb5RRGIphKNJhGMpjITj+ODTAELRqzdusFtNmFfnQI3LirpqG+b7XLq83kuzJ0kSGr0ONHodAIBE%0AMo1LgxFc6A2i2x/Gb35/ccL9HVYTFtS7sKDBhUavA3XVdvhqbKh2WmG3GhnQpAkFQ/jo0aPYsGED%0AAOD666/HyMgIQqEQXC4Xurq6UF1djaamJgDA+vXrcfToUVVDOJZIIXnF2s9JZ5pN8sW8M9Immas2%0A2X0LTWkzWEwIBOOYydy3ye4qT7N4Gdmgy2TG/pMx9n95wv9leex+8tjHmTwfF7r/2M+Qx27LyBj7%0AWva/aDyNSCwbsIOjMfgD0YLrdC1mA+bXOVFXY0ONywq3wwy3w8LApatYzEZc2+jGtY1upDMZDI3E%0AMRpJYDSSxGg4gcBoDKe7hnG6a/iq7zVIEhw2E6xmIyQJkCTAaDDAaTPBaTfDYTPBaTNnP7eZYTBc%0ADmzl9Zx7CY59YDBIue+3W03AuNeQ8prIZACjUYLZaIDZZIDJaIAEAFL2jwwJGKvn8sdQPlbuBwnj%0A/36QJAkGiwnDofhYfZdvu/K9R5bHvZ9M/N/lxxv3gTT22ZU/7/LHSl3S5e+VrngcwSyhOIKRxIy+%0Ax2YxwmxSZ85BwRAeGBjA8uXLc597vV74/X64XC74/X54vd4Jt3V1dZWm0kl0dg1j77/9ARl1J3jT%0ALJiNBrid2UB12kyoclmBjAyL2QCL2Qir2QirxQinzcQeCs2Y0WCAz2O/akJeMpXBcCiOYCSJUDT7%0AXyyeQjyZQSKZRiyRyobQWGD2ByLI8O2k4tksRuz99p2qHG8644lZc13RlG+a9mwf6z9WLyja4xER%0AEamp4BhffX09BgYGcp/39/fD5/NNeltfXx/q6+tLUCYREVH5KRjCa9euRUdHBwDg/fffR319PVwu%0AFwCgubkZoVAIFy9eRCqVwqFDh7B27drSVkxERFQmprVj1r59+3D8+HFIkoQ9e/bggw8+gNvtxsaN%0AG3Hs2DHs27cPAHDvvfdix44dJS+aiIioHKi+bSURERFlcd0HERGRIAxhIiIiQRjCRfb222/jjjvu%0AwKFDhya9/aWXXsJXv/pVPPjgg3j++edVrk6MZDKJ3bt3Y+vWrdi2bduka8mXL1+Ohx9+OPdfOp0W%0AUKm6nnzySbS1tWHLli149913J9x25MgRPPDAA2hra8PTTz8tqEJxpmqb1tZWPPTQQ7nnSl9fn6Aq%0Axejs7MSGDRvw3HPPXXVbpT9vpmobzT5vZCqa8+fPy3/2Z38mP/LII/Jvf/vbq24Ph8PyvffeK4+O%0AjsrRaFT+4he/KAcCAQGVquuXv/yl/Pjjj8uyLMuvv/66/Jd/+ZdX3ee2225Tuyyh3nrrLflb3/qW%0ALMuyfPbsWflrX/vahNu/8IUvyJcuXZLT6bS8detW+cyZMyLKFKJQ29x9991yKBQSUZpw4XBY3rZt%0Am/z9739ffvbZZ6+6vZKfN4XaRqvPG/aEi8jn8+Ef//Ef4XZPviHJyZMncfPNN8PtdsNms2H16tU4%0AceKEylWq7+jRo9i4cSMA4M4776yI37mQfNvBApiwHazBYMhtB1sppmqbSmexWLB///5J92Oo9OfN%0AVG2jZQzhIrLb7TAa8+83OjAwcNU2n36/X43ShBr/exsMBkiShERi4l6uiUQCu3fvxpYtW/Av//Iv%0AIspU1cDAADweT+7z8c+FybaDrYTniWKqtlHs2bMHW7duxb59++a8i5+emEwm2Gy2SW+r9OfNVG2j%0A0OLzhucJz9Lzzz9/1TXdv/iLv8C6deum/RhaeRIU02TtcvLkyQmfT/Z7//Vf/zW+9KUvQZIkbNu2%0ADbfeeituvvnmktaqJeX4XCiWK9tm165dWLduHaqrq7Fz5050dHRg06ZNgqojvdDq84YhPEsPPvgg%0AHnzwwRl9z2RbgK5atarYpQk1Wbs8+uij8Pv9WLp0KZLJJGRZhsVimXCfrVu35j7+3Oc+h87OzrIO%0AYW4Hm99UbQMAmzdvzn3c0tKCzs5OTbyZilbpz5tCtPq84XC0ij7zmc/gvffew+joKMLhME6cOIFb%0Ab71VdFklt3btWrzyyisAgEOHDuH222+fcPvHH3+M3bt3Q5ZlpFIpnDhxAosXLxZRqmq4HWx+U7VN%0AMBjEjh07cpczjh07VvbPlemq9OfNVLT8vOGOWUV0+PBhHDhwAB9//DG8Xi98Ph8OHjyIf/7nf8aa%0ANWvw2c9+Fq+88goOHDiQG3b90pe+JLrskkun0/j+97+PTz/9FBaLBX/7t3+LpqamCe3yd3/3d3jz%0AzTdhMBjQ2tqKb3/726LLLjluB5vfVG3zs5/9DC+++CKsViuWLVuGv/mbv6mY4y9PnTqFp556Ct3d%0A3TCZTGhoaEBrayuam5sr/nlTqG20+rxhCBMREQnC4WgiIiJBGMJERESCMISJiIgEYQgTEREJwhAm%0AIiIShJt1EFWAp59+Gq+99hpkWcb69evx53/+56JLIiIwhInK3smTJ/HrX/8a//7v/w4guzvZnXfe%0AidWrVwuujIg4HE1U5n73u9/hnnvugcVigcViwT333IPXXntNdFlEBIYwUdnr7+9HXV1d7nOfz4f+%0A/n6BFRGRgiFMVGFkWdbEdn1ExBAmKnuNjY0Ter79/f1obGwUWBERKRjCRGXurrvuwquvvop4PI54%0API5f/epXuPvuu0WXRUTg7Giisrd8+XJ8+ctfxte//nVIkoQvf/nLZX1WM5Ge8BQlIiIiQTgcTURE%0AJAhDmIiISBCGMBERkSAMYSIiIkEYwkRERIIwhImIiARhCBMREQnCECYiIhLk/wN9MgdM7CORbQAA%0AAABJRU5ErkJggg==%0A)

In [0]:
#Histogram for Std of latent variable
sns.distplot(z.describe().iloc[2,:])
plt.show()

![替代文字](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAdwAAAFYCAYAAAD5gk07AAAABHNCSVQICAgIfAhkiAAAAAlwSFlz%0AAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4yLCBo%0AdHRwOi8vbWF0cGxvdGxpYi5vcmcvNQv5yAAAIABJREFUeJzt3Xl0XfV97/3PPrOGI+lIOpps2ZZt%0AecAztgEbMODiJJgk5T6BGHpDkhtKHhpCSFva8KShcFdXU+CSrCZdfUpDQtrAc3u5EMKFMARIcAp4%0ANjZ4luUJzTqadXSkozPs5w9Zim1sSZalfab3ay0tS2fY+/vT9j4f7d/e+/czTNM0BQAAppQt0QUA%0AAJAJCFwAACxA4AIAYAECFwAACxC4AABYgMAFAMACjsleYCDQO9mLPIvPl63OztCUriOZZFJ7aWv6%0AyqT20tb0NFpb/X7vuJaRcke4Doc90SVYKpPaS1vTVya1l7amp8loa8oFLgAAqYjABQDAAgQuAAAW%0AIHABALAAgQsAgAUIXAAALEDgAgBgAQIXAAALELgAAFiAwAUAwAIELgAAFiBwAQCwwKTPFoTksHlv%0AQ8LWff3yaQlbNwAkK45wAQCwAIELAIAFCFwAACww5jnc7du36/7771d1dbUkad68eXrooYemvDAA%0AANLJuC6auuKKK/TjH/94qmsBACBt0aUMAIAFxhW4tbW1uueee3THHXfo/fffn+qaAABIO4ZpmuZo%0AL2hpadHu3bt10003qa6uTl/+8pf15ptvyuVynff10WhMDod9SorF+L2x9WTC1v2ZNbMStm4ASFZj%0AnsMtLS3Vxo0bJUkzZsxQcXGxWlpaVFlZed7Xd3aGJrfCc/j9XgUCvVO6jmQy0fb2BgemoJrxmej2%0AyaRtm0ltlTKrvbQ1PY3WVr/fO65ljNml/PLLL+tnP/uZJCkQCKi9vV2lpaUXUSYAABjzCHf9+vV6%0A4IEH9Nvf/laRSESPPPLIBbuTAQDA+Y0ZuLm5uXryySetqAUAgLTFbUEAAFiAwAUAwAIELgAAFiBw%0AAQCwAIELAIAFCFwAACxA4AIAYAECFwAACxC4AABYgMAFAMACBC4AABYgcAEAsACBCwCABQhcAAAs%0AQOACAGABAhcAAAsQuAAAWIDABQDAAgQuAAAWIHABALAAgQsAgAUIXAAALEDgAgBgAQIXAAALELgA%0AAFiAwAUAwAIELgAAFiBwAQCwAIELAIAFCFwAACxA4AIAYAECFwAACxC4AABYgMAFAMACBC4AABYg%0AcAEAsACBCwCABQhcAAAsQOACAGABAhcAAAsQuAAAWIDABQDAAgQuAAAWIHABALAAgQsAgAUIXAAA%0ALEDgAgBggXEF7sDAgG688Ua9+OKLU10PAABpaVyB+y//8i/Kz8+f6loAAEhbYwbusWPHVFtbq+uv%0Av96CcgAASE+OsV7w2GOP6aGHHtJLL700rgX6fNlyOOyXXNho/H7vlC4/2Uykvd5czxRUMj6Xsn0y%0AadtmUlulzGovbU1Pl9rWUQP3pZde0vLly1VZWTnuBXZ2hi6poLH4/V4FAr1Tuo5kMtH29gYHpqCa%0A8Zno9smkbZtJbZUyq720NT2N1tbxBvGogbt582bV1dVp8+bNam5ulsvlUllZmdauXXvx1QIAkMFG%0ADdx//Md/HPn+n/7pnzRt2jTCFgCACeA+XAAALDDmRVPD7rvvvqmsAwCAtMYRLgAAFiBwAQCwAIEL%0AAIAFCFwAACxA4AIAYAECFwAACxC4AABYgMAFAMACBC4AABYgcAEAsACBCwCABQhcAAAsQOACAGAB%0AAhcAAAsQuAAAWIDABQDAAgQuAAAWIHABALAAgQsAgAUIXAAALEDgAgBgAQIXAAALELgAAFiAwAUA%0AwAIELgAAFiBwAQCwAIELAIAFCFwAACxA4AIAYAECFwAACxC4AABYgMAFAMACBC4AABYgcAEAsACB%0ACwCABQhcAAAsQOACAGABAhcAAAsQuAAAWIDABQDAAgQuAAAWIHABALAAgQsAgAUIXAAALEDgAgBg%0AAQIXAAALOMZ6QX9/vx588EG1t7crHA7rG9/4hm644QYragMAIG2MGbjvvPOOFi9erLvvvlsNDQ36%0A2te+RuACAHCRxgzcjRs3jnzf1NSk0tLSKS0IAIB0NGbgDrv99tvV3NysJ598cirrAQAgLRmmaZrj%0AffGhQ4f013/913r55ZdlGMZ5XxONxuRw2CetQEzMG1tPJmzdn1kzK2HrBoBkNeYR7v79+1VUVKTy%0A8nItXLhQsVhMHR0dKioqOu/rOztDk17kmfx+rwKB3ildRzKZaHt7gwNTUM34THT7ZNK2zaS2SpnV%0AXtqankZrq9/vHdcyxrwtaNeuXXr66aclSW1tbQqFQvL5fBdRJgAAGDNwb7/9dnV0dOhP/uRP9PWv%0Af11/+7d/K5uN23cBALgYY3Ypezwe/eAHP7CiFgAA0haHqgAAWIDABQDAAgQuAAAWIHABALAAgQsA%0AgAUIXAAALEDgAgBgAQIXAAALELgAAFiAwAUAwAIELgAAFiBwAQCwAIELAIAFCFwAACxA4AIAYAEC%0AFwAACxC4AABYgMAFAMACBC4AABYgcAEAsACBCwCABQhcAAAsQOACAGABAhcAAAs4El0Apl4sFtfJ%0A5l519oaVn+tSQa5bBbluOR38vQUAViFw01h/OKrDH3fpaF2XBgZjZz1nsxlaOd+vBTMKZBhGgioE%0AgMxB4KaprmBYb+6o08BgTC6nTYuqCjWtOEe9oUF1BQd1oqlHOw+1KtDZrzWLyzjaBYApRuCmoeaO%0AkN7aORS2K6qLtXCWTw77UKCWFWVLkhZV+fT7vY0jXc3rV06TN9uVyLIBIK1xWJNmWjtD+h//sUf9%0A4ZhWLyzRkjlFI2F7pmyPU5++YoYWzvSpu29Qm/c0KhqLJ6BiAMgMBG4aCQ1E9D/+Y486e8NaNd+v%0AhTN9o77eZjO0emGJ5lXmq7M3rJ2HWi2qFAAyD4GbRv73O8fU3hPWZ9fO1GVVheN+3+oFJfJ53Tpa%0A363jjd1TWCEAZC4CN00c+bhT//lho6b7c/T5q6su6r12u03XLa+Q027TtgMt6gqGp6hKAMhcBG4a%0AiERj+rc3jsiQ9JWbFpz3nO1Y8nJcWrukTNGYqa37m2Wa5uQXCgAZjMBNA69sOaWWjpD+aOV0zanI%0An/ByZpZ5NaM0V4GuAdW1BiexQgAAgZvimtr79Pq2UyrMc+u/rJt9ycu7fJ5fhiHtPhJQPM5RLgBM%0AFgI3xf16y0nF4qZuX1+tLPel31adl+PSvMoC9YYiqqnvmoQKAQASgZvSWjtD2nawRdP9Obp8vn/S%0Alrt0TpEcdkMf1bYrEuXeXACYDARuCnt16ymZpvTZtbNkm8TxkLPcDi2eXaSBwZj2n+iYtOUCQCYj%0AcFNUe/eAtuxvVllhtlbNL5n05S+c6VOW267DJzs1GImN/QYAwKgI3BT12vZTisVN3bxmpmy2yZ/t%0Ax+mwaeFMnyKxuI7WMxgGAFwqAjcFdfaG9e6HTfIXeHTVotIpW091ZYEcdkOHTnVyxTIAXCICNwX9%0Adne9orG4Nl41U3bb1G1Ct9OuOdPyFRqI6lRL75StBwAyAYGbYqKxuN79qFE5HofWLi6b8vUNT4Bw%0A8GQno08BwCUgcFPMBzUB9YYiumZpuZwO+5SvLy/HpcqSXLV3DyjQ1T/l6wOAdEXgppjNexokSdct%0An2bZOi+b9YejXADAxBC4KaSxrU+HP+7Swpk+lRVmW7beEl+WivI8+rglqGB/xLL1AkA6GVfgPv74%0A49q0aZO+8IUv6M0335zqmnABm/cOHd3esMK6o1tJMgxD82YUSJKONXCLEABMxJiBu23bNh09elTP%0APfecfvrTn+r73/++FXXhHOFITFv2NSs/x6Xl1cWWr39WmVcOu6Ha+m4ungKACRhztPvVq1dr6dKl%0AkqS8vDz19/crFovJbp/6C3bwBzsPtSoUjuqzK2dOaL7bS+V02DSrPE+19d1qag+pojjH8hoAIJWN%0A+cltt9uVnT10vvCFF17QunXrCNsEePejRhmS1i2rSFgN1dOH5tpl5CkAuHjjns/t7bff1gsvvKCn%0An3561Nf5fNlyTPHtKn6/d0qXn2xiNpuO1ndr6dxiLZw7vnGTvbmeSa8jN8etwrxW1bUG5XA6Ljgd%0A4KVsn0zatpnUVimz2ktb09OltnVcgfvuu+/qySef1E9/+lN5vaOvsLMzdEkFjcXv9yoQyJxRj/x+%0Ar15795gkadU8/7jb3hscmJJ6Zld4tevwgD462qrLZhWe9zUT3T6ZtG0zqa1SZrWXtqan0do63iAe%0As0u5t7dXjz/+uP71X/9VBQUFF1chLplpmtpyoEVOh00rJ3HO24maXZEnmyEungKAizTmEe5rr72m%0Azs5Offvb3x557LHHHlNFReLOJWaSo3VdaukI6YqFJRfswrWSx+VQZalXp5p71d49oOKCrESXBAAp%0AYcxP8E2bNmnTpk1W1ILzeGd3nSRpzaKpHzd5vOZMy9Op5l4db+whcAFgnBhpKolFY3G9u7dB3myn%0AFlWd/3xpIlQU5cjjsutkcy/T9gHAOBG4SezAiQ51Bwd1xcLShNx7eyE2m6GZZV4NDMbU1N6X6HIA%0AICUkz6c4PmHrgWZJydWdPGx2RZ4k6XhjT4IrAYDUQOAmqfBgTHtr21RenKOq8uS7z6043yNvtlN1%0ArUFFovFElwMASY/ATVL7jrdrMBLXNcsqZBhGosv5BMMwVFWep2jMVF1rMNHlAEDSI3CT1K4jrZKk%0Aa5ZZOzPQxRjuVj5BtzIAjInATUKDkZg+rG1XSUGWqk6HWjLKy3GpKN+jxvY+9YejiS4HAJIagZuE%0A9h3vUDgS08oF/qTsTj7T7PI8maZ0sjkzhncDgIkicJPQcHfy6gXjm6ggkWaWDV3QdbKJwAWA0RC4%0ASSYSHbo6uTjfo5mlyXd18rmyPQ6VFWYr0NWvvv5IossBgKRF4CaZ/cc7FB6MadWCkqTvTh42a/go%0Al25lALggAjfJDHcnr5qf/N3Jw2aU5cow6FYGgNEQuEkkEo1rb22bivLcSTnYxYV4XA6VF2WrvWdA%0AvaHBRJcDAEmJwE0iB052qD8c08r5qdOdPGxW2dDtSxzlAsD5EbhJZPfh1Lk6+VwzSnNlMwzO4wLA%0ABRC4SSIai2vP0Tb5vO6kHuziQlxOuyqKs9XZG1ZjGzMIAcC5CNwkcfBkp0LhqFbNL5EtxbqTh80q%0AH/pDYcehlgRXAgDJh8BNErtOdyevWuBPcCUTV1mSK7vN0M7DrTJNJqYHgDMRuElgqDs5oIJcl+ZM%0Ay090ORPmdNg03Z+jpvYQMwgBwDkI3CRw+FSn+gaiWpnC3cnDhruVd54+YgcADCFwk0AqjZ08lmn+%0AHLlddm0/2EK3MgCcgcBNsGgsrg9q2pSf49LcFO5OHuaw27RibrHauge4RQgAzkDgJtiRui4F+yO6%0AfL5fNltqdycPW71w6Eh9+0GuVgaAYQRugo0MdpFCYyePZXFVkbLcDu083Ko43coAIInATah43NQH%0ANQF5s52aV1mQ6HImjdNh0+XzitXZG9axhu5ElwMASYHATaCj9V3qCUV0+bz06U4eduXCUknSjoNc%0ArQwAEoGbULsOBySl1lR847Vgpk+5WU7tPNKqeJxuZQAgcBMkbpraXdOqHI9D82ekT3fyMIfdplXz%0A/erpG9SRjzsTXQ4AJByBmyDHG3rUFRzUimq/HPb03Ayrh7uVGQQDAAjcRBke7CKVx04ey/zKAuXn%0AuLTrcKuisXiiywGAhCJwE8A0Te0+0qost0MLZxYmupwpY7MZWrWgRH0DUR06RbcygMxG4CbAyeZe%0AtfeEtXxukZyO9N4EV5weBIMp+wBkuvT+tE9SI93JaXh18rnmTMtXYZ5bH9S0KRKlWxlA5iJwLWaa%0ApnYfDsjtsmtRVfp2Jw+zGYZWLyhRfziq/SfaE10OACQMgWuxutagWrv6tWxOkVxOe6LLscQVw1cr%0AH+JqZQCZi8C12K4j6TvYxYXMKvPKX+DR3qNtCg/GEl0OACQEgWsh0zS163CrXA6blswuSnQ5ljEM%0AQ1deVqZwJKY9RwOJLgcAEoLAtVBjW5+aO0JaMrtIbldmdCcPW7NoqFt5y4HmBFcCAIlB4Fpo9+nu%0A5JVpPNjFhZQX5aiq3KsDJzrU3TeY6HIAwHIEroV2HWmVw27TsjnFiS4lIdYsKpNpMjE9gMxE4Fqk%0AuSOk+kCfFlcVKsvtSHQ5CXHFZaWy2wxt3U+3MoDMQ+BaZPfpwS5Wzs+87uRhedkuLa4q1KmWXjW0%0A9SW6HACwFIFrke0HW2W3GVpenZndycPWLC6TJG3j4ikAGYbAtUBDW5/qA0EtmV2kHI8z0eUk1PK5%0Axcpy27XtQLPiJhPTA8gcBK4Fhi8SuvKy0gRXkngup10r55eovSesI8wgBCCDELhTzDRN7TjYIpfT%0ApuVzM7s7edg1S8olSe9+1JTgSgDAOgTuFDvZ3KvWrn6tqPZn3GAXF1I9PV9lhdnadSSgYH8k0eUA%0AgCXGFbg1NTW68cYb9eyzz051PWlnpDt5Id3JwwzD0LplFYrG4lw8BSBjjBm4oVBIf/d3f6c1a9ZY%0AUU9aicdN7TjUomy3Q4tnp/9UfBdj7eIy2W2G/vPDRplcPAUgA4wZuC6XS0899ZRKSjJndpvJUlPX%0Apa7goFbO98thp/f+THk5Li2vLlZ9oE8nm3sTXQ4ATLkxhzxyOBxyOMY/MpLPly2HY2rPVfr93ild%0A/mR5bvMxSdKn11RdUs0Tea831zPh9V2q8db7uXVztPtIQDuOBHTF0mkX9d50kEltlTKrvbQ1PV1q%0AWyd9jMHOztBkL/Isfr9XgUDyHxFFojG9t7dB+TkuleW7J1zzRNvbGxyY0Pomw3jrne7LUmGeW5s/%0AqNcfr52pymm+lNi2kyFV/h9PlkxqL21NT6O1dbxBTD/nFNlb266+gajWLCqTzWYkupykZLMZumZJ%0AucKDMe041JrocgBgShG4U+T9fUP3mK5dUpbgSpLbumUVshmG3t5Vz8VTANLamIG7f/9+3XnnnfrV%0Ar36lX/ziF7rzzjvV1dVlRW0pqzsY1v7jHZpV5tV0f26iy0lqhXkeXT7fr/pAUPuPtye6HACYMmOe%0Aw128eLGeeeYZK2pJG1sPtChumrr69IhKGN2GVdO163CrXnn3uO6+eWGiywGAKUGX8iQzTVPv72+S%0A3WYwdvI4zZ2Wr5mlXm3f36S2rv5ElwMAU4LAnWSnWnrVEOjT8upi5WZl9sxA42UYhm5cNV1xU/rd%0ABw2JLgcApgSBO8ne3zc0VCHdyRfnioWlKsh16z8/bFR4MJbocgBg0hG4kygSjWv7wRblZTu1uIqh%0AHC+G02HTZ9bMUigc1Zb9zCIEIP0QuJNo15FWBfsjWruknKEcJ2Dj2lly2A29vv1jRWPxRJcDAJOK%0AVJhE7+wZOv94/fKKBFeSmnx5Hl27rEJt3QMjsywBQLogcCdJXWtQtfXdWlxVqBJfdqLLSVkbr5wp%0Au83Qr7ecVDzOQBgA0geBO0mGj25vuHxagitJbUX5Hl29pFwtnf3acYijXADpY9InL8hE/eGoth5o%0AVmGeW8vmFCe6nJR385qZeu+jJr2y5aSuuKxUNiN1xqLevHd8tzV5cz2TPsHE9cv5Yw9IZhzhToJt%0AB5oVHozpuuXTmKhgEvgLsrR2cZma2kPafSSQ6HIAYFIQuJfINE29s6dBdpuhdUu593ay3Lx2pgxD%0A+j/vnVAszhXLAFIfgXuJauq6VB/o0+Xz/MrPdSe6nLRR6svWtUsr1NjWp9/vbUx0OQBwyQjcS/T6%0A9o8lSRtWVya4kvTzX9bNlsdl10vvnlDfQCTR5QDAJSFwL0F9IKiPjrVr3vR8zZ2Wn+hy0k5+jkuf%0Au3qWgv0RvfzeyUSXAwCXhMC9BG+cPrr9zFUzE1xJ+rpxZaVKCrL0uw/q1dTel+hyAGDCCNwJ6ugZ%0AGg2pojhHS+cUJbqctOV02PTF9XMVi5v6X7+tlWkyGAaA1ETgTtCbO+sUi5v6zBUzUuo+0VS0orpY%0AC2f6tO94u7YeaE50OQAwIQTuBPQNRPT7DxtVkOvSVYuYZH6qGYahr960QG6XXf/fWzVq757cASMA%0AwAqMNDUBb+2sU3gwpj++uopZgSziL8jSn/xRtX7++mH97NWDeuCOFRnVsxCPmxoYjKo/HFN/OKpw%0AJKZINK5oLK7Y6TGnu4ODMgzJ5bDL7bTJ7bIrN8up/By38nNdyst2MTALkEAE7kXqCQ3qNzvrlJft%0A1PUrmBXIStcsLdeeo23aW9umt3fV61NpeCuWaZrqDUXU3jOgjp6wuoNh9YQiCoYGNdZcDh/Wto/6%0AvGFI3myX8nNc8nndKinIUmlhtkp8WSrxZakoz8MfkMAUInAv0qtbTik8GNMX1s2Wx8Wvz0qGYegr%0ANy3QsZ9t1wubj6l6er6qyvMSXdYlMU1THb1htXSE1NzRr9aOkAajZ4+s5XLaVJjnUW6WU1luh7Lc%0AdrlddjnsNjntNtnthgwZWjanSHFJkUhc4UhMA4NRBfsj6g4OqqtvUD3BsLr6BtXa1a+61uAnarEZ%0AhorzPSorylZ5UbbKi3JUVpitiuIc5WY5LfqNAOmLxLgI7d0DemdPvYrzPbqOgeITIj/Hpa9tXKgf%0Av/CRfvzCR/qbL69UcX5Wosu6KNFYXMcbunW0rlMNgaD6w7GR53KznJrmz1FRnkeF+R4V5LrlcdnH%0AtdyFswrHXUPfQEStnf2nv0Jq7exXS2e/WjpD+uhYuz46dvbRcm6W83QIDwVxeVG2yopyVJznoZsa%0AGCcC9yL8n/dOKBoz9cfXVMnpoOstUZbNLdbtf1St//jtUf3o+Y/0/3xppbI9yf1fORqLqyHQp1PN%0AvaoPBBWNDfUPe1x2zanIU1lRtsoKs5Vj0ZFkjsepqnLneXsIgv0RNbeH1NTep6aOkJrbQ2ps71Nt%0AQ7eO1nef9Vqnw6ZSX/Ynwri0MFtu5/j+UAAyRXJ/SiWRxrY+vb+/SdOKc7RmUVmiy8l4G1ZXqrWr%0AX7/dXa//96V9+vZty5Lu/KNpmmrvGVBtfY9ONPUocrqr2JvtVHWlT6U+j4rzPTKS7OKv3Cyn5k7P%0A19zpZ4+eFonG1doZUtMZYdzUPhTI9YGzu6gNDc1tXFaUrfLCHM2bVahcl03lRTnyZjuTrs2AFQjc%0AcTBNU//7nVqZpvR/rZtNF1qSuOOPqtXePaC9tW368S8/0jduWZwU59UHBqM63tij2vpudQUHJUlZ%0AbofmVRZoVplXhXlu5XmzJn0+3KnmdNg0zZ+raf7csx6Pm6a6esNqOn0kfObR8f7jHdp/vENv7aob%0AeX2Ox6GyomyVFGSpxDf879BXbhZhjPSV+E+nFPBBTUAfHWvXwpk+La9mgvlkYbMZ+r8/v0j//NI+%0A7T/eocf+5x59+7Zlys9xWV5LLB7XvuMd2rynQXWtQZmmZDOkGaW5mjs9XxVFOVP+h9rmvQ1TuvzR%0AXL98mgrzPFpUdfZ55NBARE0dIfUNxnXkZPvpMA7pZFOvjjX0fGI5WW67Sgqy5fdlqdSXJX/BH/4t%0A8LoveCtYotsOjAeBO4b+cFT/8+2jctgNfelT8/jrO8m4XXZ96wtL9Ys3jui9fU36+1/s0rduXarp%0A5xyFTZWGQFBbDjRry75mdfcNHc36vG7NnZavqgpvUhxxJ1K2x6k5Ffny+71aOss38ngsHld7T1iB%0A0xdttXT2K9A1dBFXU3ufTrX0fmJZTodNJQVD4Tt8RFziy1KpL1umabJvIull9qfBOLz07gl19ob1%0A+atnqbwoJ9Hl4Dwcdpv+28YFKsxz6+X3T+q//3ynbl4zUzevmSmnY/Iv3OnsDWv7wRZtO9Csj0/f%0AXpPtdmj95dOU5XaoMM+dcR/+Yx1henM9F+xCN2zG0EVjRdmShk7h9Iej6g1FRu5BHv63tatfDW2f%0AnMTCYTdUkOuWz/uHrwKvmwu3kFQI3FGcau7V27vrVOLL0s1rmBEomRmGoVuuna1ZZXl65s0jevn9%0Ak9pxqFWb1s/VkjlFlzwqVWggog9q2rT1QLMOn+qUKcluM7R8brGuWlSqFdXFcjrsCe3aTBeGYSjb%0A41S2x6nSc+50Mk1T4UhcvaFB9YYi6g0NqqdvUJ29YbX3DKjtnGE/vdlOFed75C/IUnFBlnxet+xc%0Ag4EEIXAvIBKN699ePyzTlO781PwpOVLC5FteXaz5Mwr04n8e1+921+tHL3yk4nyPrl8xTWsXl6kg%0A1z2u5USicdUHgjpwokP7j7ertqFH8dMzFc2dnq81l5Vq1YISebOtP1+cyQzDkMdll8c11LV8pljc%0AVE9fWJ29Q18dPWG1dw/oRFOvTjQNdVHbbYYK8zwq8XlUVpijEl8Wt/jBMgTuBfzy98d0qqVX1ywp%0A/8SFIEhuWW6H/uuGebp2abne3l2vHQdb9MLmY3ph8zEV53s0uyJPlSW5ynI75HYOjdgU7I+ouy+s%0Art5B1bUGVR8IjoxRbEiqqsjTsrnFuvKyUpUUpNZAG5nCbjPk83rk83pGHjNNUz19EbV1D50jDnQN%0AqK1r6PsDJzplM6TigqyhgTwKs1VckMURMKYMgXsee2vb9ObOOpUVZuu/bpiX6HIwQTNKvfraxoXa%0AtH6utuxv1oETHTre2KMdh1q141DrBd/nsNs0o9SrWeVeVU/P1+KqIoY2TFGGYSg/16X8XJfmTBu6%0ArzgSjSvQ1T90D3FH6PSFW/36UO1y2A2VF+Vouj9H0/y5ST+gClIL/5vO0dEzoKdfPSSH3aY/u2Wx%0A3OMcVg/JK8fj1IZVldqwqlKmaSrQPaDm9j4NDMYUHowpEoufnlXHpbwcl/wFWUk3iAYmj9NhU0Vx%0AjiqKhy6CDEdiQ2NZt4fU2B5SXWvw9FjTLSrKc2uaP1fTS3JVlIEXw2FyEbhniMbi+skrBxXsj+jO%0AT81TZYk1t5bAOoZhDA20QLcwTnM77ZpR6tWMUq8kqadvUA2BPtUHgmrpCKm9J6yPjrUr2+3QjNJc%0AzSjzqsSXlVHTQ2JyELinxU1TT792SDV1XVo136/rV3AzO5CJ8k73dCyc5dNgNKamttDIef3DH3fp%0A8Mdd8rjsQ+Fb6lU0FqdHBONgtJLcAAAN/ElEQVRC4J72y83HtO1Ai+ZMy9Ndn72MriMAcjnsmlnm%0A1cwyr+JxU80dIZ1q7lVda1A1dd2qqevW1v3NWlHt16oFfi2cyQWWuDACV9Jbu+r0+vaPVVaYrftv%0AXcbN8gA+wWYzRs79Xmmaau3o16mWXjV3hPTevia9t69JWW67rlxUrkUzfVoyu1AuPktwhowP3N99%0AUK//9fZR5ee49BdfXMbVqADGZDP+MDrWumUVOt7Qo11HWrX7SKs2f1CvzR/Uy+20a8nsQq2Y59ey%0AOUXK9vDZkukyNnBN09RL757QK1tOKi/Hpb/YtFzFXEgD4CLZDGNkOsNN6+eqOxzTW9tOaveRgHad%0A/rLbDC2Y6dPl8/xaUV087gFYkF4yMnBj8biefbNGv9/bqJKCLP3FpmUq8WUnuiwAKc4wDFVX+lTg%0AcejW6+aooa1PH9QEtKemTQdOdOjAiQ4985sjmlORp8vn+XX5PL9KC/nsyRQZF7jt3QP6ySsHdLS+%0AWzNKc/XnX1yekOncAKQ3wzA03Z+r6f5cff7qKrV192tPTZs+qAmopr5Lxxp79PzmY6ooztGS2YVa%0AMrtI1dMLGGoyjWVU4O463Kp/e/2wQuGoVs33679tXKgsd0b9CtIekwcgWRXnZ2nD6kptWF2pntCg%0APqxtGzryPdmh3+yo02921MnltGnhDJ8Wzy7SktmF9LylmYxIm7aufj2/+Zh2Hm6Vy2nTV29aoGuX%0AlnPrD4CEyMt26dqlFbp2aYUGIzHV1HVp3/EO7T/Rrg+PDX1JUlGeW/MqfZo/o0DzZxSopCCLz60U%0AltaB2x+O6tWtp/TmzjpFY3HNrsjTXTcvZF5bAEnD5bRr8ewiLZ5dJKlabV392n+iQ/tPdOjIx53a%0AeqBZWw80S5IKcl2aV1mg6ukFqiofmoSDLujUkZaB29rVr9/uqtd7+xrVH47J53Xr1uvn6MrLShmO%0ADcCkOvc0hjfXo97gwAVePU6GtHh2oRZV+dQVHFRLR0gtnf1q6QidNfmGzZAqS72qKvNqVnmeZpV5%0AVV6UQwgnqbQJ3NBARHuOtmnn4VbtO9YuU1J+jksbr5qpG1dVMpgFgJRjGIZ8Xrd8XrcWzPSdnm5w%0AUIGuAbX3DKite0ANgaBONfdKexslDd2mVFqYpYriHE07PVDHtOIclfiyCeIEG1fgfv/739eHH34o%0AwzD03e9+V0uXLp3qusYUjcV1srlXNXVdOnyqU4dOdY7MX1pV7tWNqyq1ekEJY5wmwEQvXJqUIwMg%0AjQ1NN+hWfq5bczU03eA1S8pVHwjqRFOvTjX3qKGtT41tfWpqD2n3kcAf3ivJl+eWPz9L/oIs+Qs8%0A8hdkqSjfI9/pZRLIU2vMwN2xY4dOnTql5557TseOHdN3v/tdPffcc1bUJmlo6qy27qFJo1s6+9Xe%0AG1ZtXacaAn0ajMZHXjejNFer5pdo5Xw/52gBZAyH3aZZZXmaVZYnaWjSFdM01dkbVmN7nxoDfWpo%0A61NrZ78C3f2qqevSkbqu8y4rN8upgly3CrwuFeS65c1yKjfLqZwsp3I8DuV4zv4ZF2fM39jWrVt1%0A4403SpLmzJmj7u5uBYNB5eZO/dR1v/z9Mb269dQnHrefHtN07vR8za8s0LzKAkZuAYDTDMNQYZ5H%0AhXkeLa4qOuu5SDSu9p4BBbr6FejqV3v3gLqCYXUFB9XZG1Zbd7/qA8FxrcfltCvLbZfbOfTlctpG%0Avh/6+fT3LpscdpvsNkN2m012uyGHzZB95LGzvx9+7XguuYmbQ4MZxWKmYvHTX7G4ojFT4UhMA4NR%0ADQzGNBA+/X0kplXzS3TlZaUT+dVekjEDt62tTYsWLRr5ubCwUIFAwJLALSvM1qKqQhXleVSc71GJ%0AL0uL55XIJZOuYgCYAKfDprLCbJWNMsJVfziq7r5BBfsj6uuPDP07EFVff0R9A0PfB/sjCkfj6gmG%0AFY7E1BsaVHgwrrhpWtiaifE47ckZuOcyx/hl+v3eCRdzrlvWe3XL+nmTtrxUNZHf6W0bFkxBJQCQ%0AuS4138Y8TCwpKVFbW9vIz62trfL7/Ze0UgAAMs2YgXv11VfrN7/5jSTpwIEDKikpsaQ7GQCAdDJm%0Al/Lll1+uRYsW6fbbb5dhGHr44YetqAsAgLRimGOdlAUAAJeMS30BALAAgQsAgAWScqiQSCSiBx98%0AUI2NjbLb7fqHf/gHVVZWnvWa1157TU8//bRsNpvWrFmjP//zP9eLL76oH/3oR5oxY4Ykae3atfqz%0AP/uzRDRhXEYbMnPLli364Q9/KLvdrnXr1unee+8d8z3JbLS6t23bph/+8Iey2WyqqqrS3//932vn%0Azp26//77VV1dLUmaN2+eHnrooUSVf9FGa+/69etVVlYmu31ofO8nnnhCpaWlabdtW1pa9MADD4y8%0Arq6uTn/5l3+pSCSSUvvpuWpqavSNb3xDX/3qV/WlL33prOfSbb8dra3ptt+O1tZJ22fNJPTiiy+a%0AjzzyiGmapvnuu++a999//1nPh0Ih84YbbjB7e3vNeDxu3nrrrebRo0fNX/7yl+ajjz6aiJIv2vbt%0A282vf/3rpmmaZm1trfnFL37xrOdvuukms7Gx0YzFYuYdd9xhHj16dMz3JKux6t6wYYPZ1NRkmqZp%0A3nfffebmzZvNbdu2mffdd5/ltU6Gsdp7ww03mMFg8KLek6zGW3ckEjFvv/12MxgMptR+eq6+vj7z%0AS1/6kvm9733PfOaZZz7xfDrtt2O1NZ3227HaOln7bFJ2KW/dulUbNmyQNPTX7wcffHDW81lZWXr5%0A5ZeVm5srwzBUUFCgrq7zjw2arC40ZKY0dCSQn5+v8vJy2Ww2XXfdddq6deuo70lmY9X94osvqqys%0ATNLQSGadnZ0JqXOyTGQ7peu2HfarX/1Kn/70p5WTk9rjnLtcLj311FMqKSn5xHPptt+O1lYpvfbb%0Asdp6PhPZrkkZuG1tbSosLJQk2Ww2GYahwcHBs14zfC/wkSNH1NDQoGXLlkkammzhrrvu0le+8hUd%0APHjQ2sIvQltbm3w+38jPw0NmSlIgEBhp/5nPjfaeZDZW3cPbsrW1Ve+//76uu+46SVJtba3uuece%0A3XHHHXr//fetLfoSjGc7Pfzww7rjjjv0xBNPyDTNtN22w55//nndeuutIz+nyn56LofDIY/Hc97n%0A0m2/Ha2tUnrtt2O1VZqcfTbh53Cff/55Pf/882c99uGHH571s3mBO5dOnjypBx54QD/4wQ/kdDq1%0AbNkyFRYW6vrrr9eePXv0ne98R6+88sqU1T6ZLtTGyX5PMjhf3e3t7brnnnv08MMPy+fzadasWfrm%0AN7+pm266SXV1dfryl7+sN998Uy6XKwEVX5pz2/utb31L1157rfLz83XvvfeODCwz2ntSxfnq3rNn%0Aj2bPnj3yAZ3K++lkSNVtez7pvN+eabL22YQH7m233abbbrvtrMcefPBBBQIBLViwQJFIRKZpfmKD%0ANTc3695779Xjjz+uhQsXSho6rJ8zZ44kacWKFero6FAsFhs50Z1MRhsy89znWlpaVFJSIqfTmZLD%0AbI41PGgwGNTdd9+tb3/727rmmmskSaWlpdq4caMkacaMGSouLlZLS8snLp5LRmO195Zbbhn5ft26%0AdaqpqUnZIVTHU/fmzZu1Zs2akZ9TaT+9GOm2344l3fbb0UzWPpuUXcpXX3213njjDUnSO++8oyuv%0AvPITr/mbv/kbPfLII2fNZPTUU0/p17/+taShK84KCwuTdicebcjM6dOnKxgMqr6+XtFoVO+8846u%0AvvrqlB1mc6y6H330UX3lK1/RunXrRh57+eWX9bOf/UzSUFdde3u7Skutn91jIkZrb29vr+66666R%0AUyQ7d+5UdXV12m5bSdq3b58WLPjDZBqptJ9ejHTbb8eSbvvthUzmPpuUI03FYjF973vf08mTJ+Vy%0AufToo4+qvLxcP/nJT7R69WoVFBTolltuOesS7K9+9atatGiR/uqv/kqmaSoajSb95fdPPPGEdu3a%0ANTJk5sGDB+X1erVhwwbt3LlTTzzxhCTpU5/6lO66667zvufMD7JkdqG2XnPNNVq9erVWrFgx8trP%0Afvazuvnmm/XAAw+op6dHkUhE3/zmN0fOEaWC0bbtv//7v+ull16S2+3WZZddpoceekiGYaTdth2+%0A8PFzn/ucfv7zn6u4uFjSUO9UKu2nZ9q/f78ee+wxNTQ0yOFwqLS0VOvXr9f06dPTbr8dra3ptt+O%0AtV0na59NysAFACDdJGWXMgAA6YbABQDAAgQuAAAWIHABALAAgQsAgAUIXCDF1dbW6sCBA594PBqN%0Aav78+QmoCMD5ELhAinvrrbdSajxiIFMlfGhHAON35hyzAwMDWr9+vZ599lnl5ubK4/GMDP6SlZV1%0A3hHaACQOR7hACnn99dc1e/ZsPfPMM3r22Wfl9Xp17bXX6k//9E/1uc99Tv/8z/+sL3zhC3r22Wfp%0ATgaSDIELpJBrr71WW7du1YMPPqjf/e532rRp01nP19TUaOXKlZKkq666KhElArgAAhdIIXPmzNGr%0Ar76qz3/+89q6davuvPPOs543TVM229BuHYvFElEigAsgcIEU8sorr2jfvn1au3atHn74YTU1Nckw%0ADEUiEUlDgbx3715J0tatWxNZKoBzcNEUkELmzp2rhx9+WC6XS6Zp6u6775bX69Xjjz8u0zR17733%0A6jvf+Y7eeOMNrVixQg4HuziQLJgtCAAAC9ClDACABQhcAAAsQOACAGABAhcAAAsQuAAAWIDABQDA%0AAgQuAAAWIHABALDA/w+N6vuK+emMEAAAAABJRU5ErkJggg==%0A)

In [0]:
#Histogram for mean of latent variable
sns.distplot(z.describe().iloc[1,:])
plt.show()

![替代文字](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAd8AAAFYCAYAAAAStfY4AAAABHNCSVQICAgIfAhkiAAAAAlwSFlz%0AAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4yLCBo%0AdHRwOi8vbWF0cGxvdGxpYi5vcmcvNQv5yAAAIABJREFUeJzt3Xl0m/WdLvDnfbXvkmV5jx1ngYSE%0AkEBZAmEbSJcp7e3G0t7Sck5v5/aUk0Ln9Awc2hm4ZaBtyjBDW+7QgdKeA8OcTGno0FuWAkNaWpJA%0AyEJCNseJHe+WZO279L73D1lOQuJNlvS+kp7POTmJY1n6+rWsR79dkGVZBhEREVWMqHQBRERE9Ybh%0AS0REVGEMXyIiogpj+BIREVUYw5eIiKjCGL5EREQVpq3Eg3i9kUo8TNFcLjMCgbjSZdQEXsvS4HUs%0ADV7H0uG1nD+Pxzbt59jyBaDVapQuoWbwWpYGr2Np8DqWDq9laTF8iYiIKozhS0REVGEMXyIiogpj%0A+BIREVUYw5eIiKjCGL5EREQVxvAlIiKqMIYvERFRhTF8iYiIKozhS0REVGEMXyIiogpj+BIREVVY%0ARU41IqLZbds7NPVvm9WISDQ5p6+7bm17uUoiojJhy5eIiKjC5hS+R48exY033ohnn30WADAyMoLb%0Ab78dX/rSl3DXXXchnU6XtUgiIqJaMmv4xuNxPPjgg1i/fv3U//3kJz/Bl770JTz33HPo6urC888/%0AX9YiiYiIasms4avX6/Hkk0+iqalp6v927tyJG264AQBw/fXXY/v27eWrkIiIqMbMOuFKq9VCqz3z%0AZolEAnq9HgDgdrvh9XpnvA+XywytVrOAMsvP47EpXULN4LUsjs1qnPHj6fB6z4zXp3R4LUtnwbOd%0AZVme9TaBQHyhD1NWHo8NXm9E6TJqAq9l8U6f3Tyf2c683tPj87F0eC3nb6Y3K0XNdjabzUgm8y8M%0AY2NjZ3RJExER0cyKCt8rr7wSr776KgDgD3/4A66++uqSFkVERFTLZu12PnDgAH70ox9haGgIWq0W%0Ar776Kh555BHce++92LJlC9ra2vCZz3ymErUS1R1JlpFM5aARBRj06p43QURzN2v4rl69Gs8888xZ%0A///LX/6yLAUR1TtZlrF9/wgO9fmRTOUgAxAFYMNFbVjcwgkvRLWAO1wRqcyh/gB2HxmHJAEelwld%0ALTZoRBFv7R3G8eGQ0uURUQlwb2ciFRn2xfDeYS/MRi3++opOmI06AIAvmMDruwbx5/dHkZOA5R0O%0AhSslooVgy5dIJcKxNP60bxiCIOAT6xdPBS8ANDpN2HjZIhh0Gmw/MIphX0zBSolooRi+RCogSTK2%0A7RlCOiPhilXNaHFbzrqN227EDR/pAAC8d8Q7pzX2RKRODF8iFdh3zIdgNI0lbXYsm6FLudFhRHer%0ADYFICv2j3PCAqFoxfIlU4LVdAwCA1d0Ns972omWNEARgb48PksTWL1E1YvgSKWxgPIrDJ4NodZvh%0AtBlmvb3dosfyDgfC8Qx6h8MVqJCISo3hS6SwQqt3ZZdrzl+zZqkboihg3zEfMlmpXKURUZkwfIkU%0AFI6nseODMTQ5TWj3nD3Jajpmow4rOp2IJ7P4077hMlZIROXA8CVS0B/3DiObk3DDJR0QBGFeX7uq%0AuwGCAIYvURVi+BIpJJuT8ObuQRj1GmxY0zrvrzcZtOjwWDEwHsXJMc58JqomDF8ihRzsm0AwmsaV%0Aq1tgMhS32dzSdjsA4M/7R0pZGhGVGcOXSCF7enwAgMtWNhd9Hx0eK2xmHXZ8MIZsjhOviKoFw5dI%0AAZIsY2+PD1aTDsvai9+nWRQFXH5BM6KJDPb3+ktYIRGVE8OXSAEnhsMIxdJYu6wRoji/iVYftuHC%0A/Hgxu56JqgfDl0gBu3u8AIB15zUu+L46m21Y1GTF+71+hOPpBd8fEZUfw5dIAXt7fNDrRKxaPPt2%0AknNx1eoW5CQZOz8YK8n9EVF5MXyJKmzEH8OIP45Vixug12lKcp9XrGqBKAjYcZDhS1QNGL5EFbZ3%0Acpbzxed5Snafdose5y1y4MRIGMFoqmT3S0TlwfAlqrDdPV4IQn5/5lJauzwf5vuO+Up6v0RUegxf%0AogoKRVM4PhTGeR1O2Mz6kt732mX5MC+0rIlIvYrbVoeIivJ+rx8ygHXLFz7LuWDb3qGpfzutehw4%0AMYHXdg1Ap539vfV1a9tLVgcRzR1bvkQVdKg/AABYvaS0Xc4FHU1W5CQZI/5YWe6fiEqD4UtUIbIs%0A41B/AA6LHq1uc1keY1GTFQAwMB4ty/0TUWkwfIkqZMQfRyiWxsou17yPD5yrRocRRr0GQ94YJFku%0Ay2MQ0cIxfIkqpNDlvKLLVbbHEAQBHU1WJNM5+ILJsj0OES0Mw5eoQgrhu7KM4Quw65moGjB8iSpA%0AkmQcORlAo8MIj9NU1sdqdZuhEQUMMnyJVIvhS1QBA+NRxJLZsnY5F2g1IlrcZoRiacQSmbI/HhHN%0AH8OXqAIq1eVcUJhNPeKPV+TxiGh+GL5EFTA12aqzMuHb5rYAANf7EqkUw5eozLI5CUcHgmh1m+Gy%0AGSrymA6rHiaDBiP+OGQuOSJSHYYvUZn1jUSQyuQq1uUM5JcctbotSKZzPOWISIUYvkRldqh/AkDl%0AxnsLpsZ9fRz3JVIbhi9RmR0dCAIAzlvkrOjjctIVkXoxfInKKCdJODYcRqvbXPIjBGdjNurgsOox%0AFogjJ0kVfWwimhnDl6iMBsdjSKVzWN5R2VZvQavbjGxOhpdbTRKpCsOXqIyODua7nJd3OBR5/Nap%0AJUfseiZSE4YvURn1DIYAAMsrPN5b0NJghiAAIz6u9yVSE4YvUZnIsoyewSAcVj08DqMiNei0Ihod%0AJvhDSaQzOUVqIKKzMXyJysQbSiIUTWN5h7Ns5/fORavbDBnAeCChWA1EdCaGL1GZ9AwoO95b0OTK%0An6I0xvAlUg2GL1GZFMZ7z1NopnOBx2mCIABjE5x0RaQWDF+iMukZDMKg16CjyaJoHTqtCLfdCH84%0AiUyW632J1IDhS1QGkXgaI/44lrXZoRGV/zVrbjBBlgFvkF3PRGqg/KsCUQ06NjS5xEjhLueCZld+%0Aq0lOuiJSB4YvURlMre9VeLJVwdSkK477EqmCtpgvisViuOeeexAKhZDJZHDnnXfi6quvLnVtRFWr%0AZzAIURCwpE0d4avXaeCyGeANJZGTJFV0hRPVs6J+A1944QV0d3fjmWeewWOPPYaHHnqo1HURVa10%0AJoe+kQi6Wqww6DVKlzOlucEESZLhC3GfZyKlFRW+LpcLwWB+DWM4HIbLVdlzSonU7MRIGDlJVs14%0Ab0Fh3HdsguO+REorqtv5k5/8JLZu3YqNGzciHA7j5z//+Yy3d7nM0GrV0wI4F4/HpnQJNaPer+Wb%0A+0YAAJdc0DKva2GzGmf8eKGWdGjwx73D8IeTU/ddDz+revgeK4XXsnSKCt//+q//QltbG37xi1/g%0A8OHDuO+++7B169Zpbx8IqHuSh8djg9cbUbqMmsBrCew9Mg4AaLIb5nUtItFT3cE2q/GMj0vFYdFj%0AxBdDKJyAKAo1/7Pi87F0eC3nb6Y3K0WF7+7du7FhwwYAwIoVKzA+Po5cLgeNRt2tW6Jy++89gzh8%0AMgCbWYc9PV6lyzlLk8uEnsE0JsJJNDpNSpdDVLeKGvPt6urCvn37AABDQ0OwWCwMXiIAoWgKmaw0%0AtbRHbZob8nWNc7MNIkUV1fK99dZbcd999+HLX/4ystksHnjggRKXRVSdCocXNE1OblIbz2Rr1xvk%0AjGciJRUVvhaLBY899lipayGqeoUdpJpV2vK1mnQw6jXcZpJIYVxpT1RC44EEjHoNbGad0qWckyAI%0A8DhNiCeziCUySpdDVLcYvkQl4g8lEU9m0eQyQRAEpcuZlseZX2bE1i+Rchi+RCXSM5jfeKZJ5bOI%0APS6O+xIpjeFLVCKFwxTUOtO5wG03QhTY8iVSEsOXqER6BoPQiAIa7KXdmarUtBoRDXYj/OEk0pmc%0A0uUQ1SWGL1EJxJIZDHljaHQaIYrqHe8t8DhNkGWgb5Q7FhEpgeFLVAI9AyHIOHV4gdoVxn17h0IK%0AV0JUnxi+RCVwdHKyVWEHKbUrzHg+xvAlUgTDl6gEjg7kx3sbHdURvhajDmajFr1DIciyrHQ5RHWH%0A4Uu0QKl0Dv2jEXS12KDTVs+vlMdpQjiegTfEJUdElVY9rxREKtU7HEJOknHeIqfSpcxLYT1y7yC7%0AnokqjeFLtEBHB/LjvdUWvlPjvsMMX6JKY/gSLdDRgSAEAMs7HEqXMi8uuwFajYC+kbDSpRDVHYYv%0A0QJkcxJ6h8No91hhMarzMIXpaEQRi5psODkWRSYrKV0OUV1h+BItQN9IBJmshPOrrMu5YEmrHTlJ%0AxsB4VOlSiOoKw5doAQrre8/rrM7w7W6zAQBOsOuZqKIYvkQLMDXZqsrGewu6W+0AgOPDDF+iSmL4%0AEhVJkmT0DIbQ7DLBYTUoXU5RmhvMMBm0bPkSVRjDl6hIA+NRJFJZLK/S8V4AEAUB3a02jE7EEU9m%0AlC6HqG4wfImKdKg/AABY2eVSuJKFKXQ9nxjhCUdElcLwJSrS4ZP58F3RWd3hu6Qw7suuZ6KKYfgS%0AFSGbk3BkIIhWtxkuW3WO9xZ0t022fDnpiqhiGL5ERegbjSCVzmFFlXc5A4DTaoDLZsDxkTBPOCKq%0AEIYvURGmxnurvMu5YEmrHeFYGoFISulSiOoCw5eoCIcnw7cWWr7Aqa5nrvclqgyGL9E8pTM59AyG%0A0NlkhdVUXfs5T+fUjGeGL1ElMHyJ5ql3KIRsTqqZVi8ALG6xQQDDl6hSGL5E83RoconRBYtrJ3xN%0ABi2aG8zoH4tA4qQrorLTKl0AUbU51B+AKAhY3lG9O1sVbNs7NPVvo0GD0Ykc/t/bfbBb9DN+3XVr%0A28tdGlFNY8uXaB4SqSxODEfQ3WaDyVBb713ddiMAYCKcVLgSotrH8CWah6MDQUiyXPVbSp5LIXz9%0AYS43Iio3hi/RPHxwYgIAsLKrQeFKSq/Bnt+py8+WL1HZMXyJ5uHAiQkY9Bosr9Lze2ei12lgM+sw%0AEU5ypyuiMmP4Es2RL5jA6EQcKztd0Gpq81fHbTcinZEQTfB4QaJyqs1XEKIyODDZ5bx6Se11ORcU%0Aup4nOO5LVFYMX6I52n/cDwBYvcStcCXl43YUJl1x3JeonGprrQTRaU5fwzpfH17Hms1JONQfQJPL%0AhCanaaGlqVaDbTJ8QwxfonJiy5doDnqHQkimc7iwu3ZbvQBg0GtgNekwEU5x0hVRGTF8ieagMN67%0AqobHewvcdgNSmRxiyazSpRDVLIYv0RwcOD4BrUbAis7q31JyNg3c6Yqo7Bi+RLMIx9LoH4tgeYcT%0ARn3tT5M4NemKM56JyoXhSzSLD+pgidHpppYbcdIVUdkwfIlmMbXEqMYnWxUY9VpYjFr4udMVUdkw%0AfIlmkJMk7D/uh8tmQIfHonQ5FeN2GJFM55BIcdIVUTnU/gAWUREKa4THAnHEklm0eyz4475hhauq%0AnAa7ESfHovCHUzAbdUqXQ1Rz2PIlmsHQeAwA0OGxKlxJZbkLJxxx3JeoLBi+RDMY9EahEQW0uM1K%0Al1JRXG5EVF5Fh++LL76IT3/60/jc5z6Hbdu2lbAkInWIJjIIRtNocZtr9hSj6ZgMWpgNWi43IiqT%0Aol5RAoEAHn/8cTz33HN44okn8MYbb5S6LiLFDXqjAFBXE61O1+AwIpHKctIVURkUFb7bt2/H+vXr%0AYbVa0dTUhAcffLDUdREprjDe215n470FU+O+7HomKrmiZjsPDg4imUziG9/4BsLhMDZt2oT169dP%0Ae3uXywytVlN0kZXg8diULqFmqOVa2qzGor82k81hdCIOt8OIVoW+n4XUXwodTXbsO+ZHNJE7qxa1%0A/IznoppqVTtey9IpeqlRMBjEz372MwwPD+MrX/kK3nzzTQiCcM7bBgLxogusBI/HBq83onQZNUFN%0A1zISLb7FNjAeRU6S0eo2L+h+imWzGhV53NOZ9PmOsRFfFJGo44zPqeVnPBs1PR+rHa/l/M30ZqWo%0Abme3241169ZBq9Wis7MTFosFExMTRRdIpDaD44Xx3vrscgYAs1ELk0HDbmeiMigqfDds2IAdO3ZA%0AkiQEAgHE43G4XK5S10akCFmWMeSNwaDToNGpbNev0hrsRsSTWSTTnHRFVEpFdTs3NzfjYx/7GG65%0A5RYAwPe+9z2IYn0txaDaFYikEE9lsaTNDnGaoZR64bYbMeSNwR9Kod3DDfGISqXo36bbbrsNt912%0AWylrIVKFQW9hlnN9LjE63dQJR+EkrwdRCbG5SvQhg+NRCALQ3siwOXW2L8d9iUqJ4Ut0mkQqC18o%0AiSanCXqdupfHVYLZoIVRr8EEd7oiKimGL9Fphn2TByk01e8s59MJgoAGuxHRRAbJdE7pcohqBsOX%0A6DQc7z1bYaerQIRdz0SlwvAlmiRJMoZ9MVhNOjgseqXLUY1TJxyx65moVBi+RJPGAwlkshI6PJZp%0Ad2urRw3c45mo5Bi+RJOmTjHieO8ZrCYddFoRAbZ8iUqG4Us0aXA8Cq1GQHODSelSVEUQBDTYDAjF%0A0shkJaXLIaoJDF8iAJF4GuF4Bi1uCzTcre0shXHfYIStX6JS4KsMEU4tMeLGGufGcV+i0mL4EgEY%0A8uWPvWxrNCtciTpNzXhmy5eoJBi+VPdykoxRfwx2ix42M5cYnYvDoodGFDDBli9RSTB8qe55Awlk%0AczK7nGcgigKcNgOCkTRykqx0OURVj+FLdW9ocry3jeE7I7fdAEmWEYqy65looRi+VPeGfTFoRC4x%0Amk2DjTtdEZUKw5fqWjyZRSCSQpPLBK2Gvw4zOf1sXyJaGL7aUF2bWmLEgxRm5bQZIAic8UxUCgxf%0AqmvDHO+dM61GhMOix0Q4CUnmpCuihWD4Ut2SZBnD/hgsRi1PMZqjBrsR2ZyM8UBC6VKIqhrDl+rW%0ARCiJdEZCWyNPMZqrwrjvybGIwpUQVTeGL9WtkYn8rlYtbu5qNVeFGc/9DF+iBWH4Ut0a9U+GbwPD%0Ad66mWr6jDF+ihWD4Ul3KSflxS6dVD5NBq3Q5VUOv08Bq0qF/LAqZk66IisbwpbrkCyaQk2R2OReh%0AwW5ANJFBgEuOiIrG8KW6NDrBLudiFU444rgvUfEYvlSXCuO9zQzfeXNPzXiOKlwJUfVi+FLdyeYk%0AeINJuO0GGHQapcupOoWWL5cbERWP4Ut1ZzyQgCRzvLdYJkN+UxKGL1HxGL5Udzjeu3CdzTb4wylE%0AExmlSyGqSgxfqjuj/jgEAWhyMXyL1dViBcBJV0TFYvhSXUlnc/CHk2h0GKHT8ulfrM4mGwCO+xIV%0Ai68+VFfGJxKQZaDFzVOMFqKzpRC+nPFMVAyGL9WVscnTeJpdJoUrqW4ehxEmgxb93GaSqCgMX6or%0A44EEBAAeJ8N3IQRBQFezFWMTcSTTWaXLIao6DF+qG7mcBH8oCZfdwPHeEuhstkEGMDgeU7oUoqrD%0AVyCqG/5wEpIso4ldziXR2cwZz0TFYvhS3RifHO/lEqPS6GrOT7riuC/R/DF8qW5MhS/He0ui1W2B%0AXieij+FLNG8MX6oLsixjPJiA1aSD2cjze0tBFAV0Ntkw7IshnckpXQ5RVWH4Ul0IxdJIZySO95bY%0A4hYbJFnGwDjX+xLNB8OX6sKp8V6Gbyl1TW62wa5novlh+FJd4HhveSxu4aQromIwfKkujAcS0OtE%0AOKx6pUupKZx0RVQchi/VvHgyi2gigyanCYIgKF1OTeGkK6LiMHyp5o0HOd5bTl2cdEU0bwxfqnne%0AyfFeD8O3LBZz0hXRvC0ofJPJJG688UZs3bq1VPUQlZw3mIAgAG67UelSahInXRHN34LC91//9V/h%0AcDhKVQtRyeUkGRORFFw2A7QadvSUw6lJV2GlSyGqGkW/GvX29uLYsWO47rrrSlgOUWkFIylIkoxG%0AB1u95XJq0lWck66I5qjo8P3Rj36Ee++9t5S1EJWcN5Qf7210cLy3nDjpimh+itrk9re//S3Wrl2L%0ARYsWzen2LpcZWq2mmIeqGI/HpnQJNUMt19JmNSIcywAAOlsdsFmrq/Wr5no//DNec54Hb7w3CH80%0AjStU8vMvUMvzsRbwWpZOUeG7bds2DAwMYNu2bRgdHYVer0dLSwuuvPLKc94+EIgvqMhy83hs8Ho5%0AWaQU1HQtI9EkRnwx6DQitKKMSDSpdElzZrMaVV3vh3/GDWYdAGD/MS8uO9+jREnnpKbnY7XjtZy/%0Amd6sFBW+//Iv/zL175/+9Kdob2+fNniJlJLO5BCKpdHSYIbIzTXKqtVtgUGnQd8IX5yJ5oLTP6lm%0A+cP5liMnW5WfKApY3JLf6SqRyipdDpHqLfhg002bNpWiDqKS8wUnw9fJ8K2E7jY7jgwE0Tcawcou%0Al9LlEKkaW75Us3whtnwraUmrHQBwYoTrfYlmw/ClmiTLMnyhBMwGLcxGndLl1IUlbfnwPT7M8CWa%0ADcOXalIgkkIilWOXcwW5bAY4rHq2fInmgOFLNanQ+mKXc+UIgoAlrXYEIikEIimlyyFSNYYv1aTj%0AI4Xw5c5WlcSuZ6K5YfhSTTox+eLf4DAoXEl96Z6cdHV8JKRwJUTqxvClmiNJMvpGI3Ba9dCrfFvT%0AWrO4xQ4Bp978ENG5MXyp5gz7YkhlcnBzvLfizEYtWtxmnBiNQJJkpcshUi2GL9Wcwnivh+O9iljS%0AZkcqncOIP6Z0KUSqxfClmlOY7OPmMiNFFDbb4KQroukxfKnmnBgJQ68V4bJyspUSlrQ5AHCnK6KZ%0AMHyppqTSOQx6o+hssUEUeZKREto9Fui0Ilu+RDNY8MEKRGrSPxaBLJ/q+qTy2LZ3aMbPu2wGDIxH%0A8dquAei0Z77Hv25tezlLI6oKbPlSTSm0tgqbPZAympwmyAC8wYTSpRCpEsOXakphpnM3W76KanLl%0AZ5qPBxi+ROfC8KWacmI4DJtZxz2dFeZxToYvW75E58TwpZoRiqXhDyfR3WqHIHCylZIMeg2cVj18%0AwQQ32yA6B4Yv1YzCloacbKUOHqcJ2ZzME46IzoHhSzWjsJk/J1upA8d9iabH8KWaUWj5LmbLVxVO%0AhW9c4UqI1IfhSzVBkmWcGImg2WWC1aRTuhwCYDXpYDJoMB5MQJY57kt0OoYv1YSxiTjiqSy62eWs%0AGoIgoMllRiKVQzSRUbocIlVh+FJNKOwjzMlW6tLk5Lgv0bkwfKkmFHa2YstXXTjpiujcGL5UE06M%0AhKERBXQ2WZUuhU7jshmg1QjcbIPoQxi+VPUy2RxOjkXR2WyFTqtRuhw6jSgK8DhNCEXTSKazSpdD%0ApBoMX6p6J8ejyEky93NWqeYGMwBgdIKtX6IChi9VvRM8yUjVWt358B3xxRSuhEg9GL5U9XiSkbq5%0A7UbotCJGJ7jZBlEBw5eq3onhMMwG7VT3JqmLKApobjAjEs8gGud6XyKA4UtVLprIYCyQQHerDSJP%0AMlKtqa7nCXY9EwEMX6pyfSNc31sNpsLXz65nIoDhS1Xu+NTOVg6FK6GZOCx6mAwajPrj3OeZCAxf%0AqnLc2ao6CIKAVrcFyXQOQ152PRMxfKlqybKMEyNhuO1GOCx6pcuhWbRMTog72B9QuBIi5TF8qWr5%0AQklE4hm2eqtEYdz3UN+EwpUQKY/hS1WrdygEAFjK8K0KFpMOdrMOhweCyOYkpcshUhTDl6pW71B+%0AvHdpOydbVYsWtwWpdG5qrJ6oXjF8qWodGwpBqxHQ1WxTuhSao3aPBQCwr9encCVEymL4UlVKpXMY%0AGI+iq8UGnZZP42rR6jZDrxWxt4fhS/WNr1pUlfpGw5BkGUvb2OVcTbQaERcsbsCIP44x7vVMdYzh%0AS1Xp2ORkq2Uc760665Y3AgD2HmPrl+oXw5eqEidbVa81yxohAOx6prrG8KWqI8syeodDaLAb4LIZ%0AlC6H5slh0WNJux1HB4OIJnjKEdUnhi9VHW8wgUg8w/HeKrZ2WSNkGXifs56pTjF8qeoUupw53lu9%0A1i73AGDXM9Uvhi9VnWPDkztbMXyrVpvbjCanCftPTCCT5W5XVH+KDt/Nmzfj1ltvxec//3n84Q9/%0AKGVNRDPqHQxBpxXR2WxVuhQqkiAIWLu8Eal0DodP8qAFqj9Fhe+OHTvQ09ODLVu24KmnnsLDDz9c%0A6rqIzimZzmLAm99cQ6thx001Kyw5evfQuMKVEFVeUa9el156KR577DEAgN1uRyKRQC6XK2lhROdy%0AYiQCWQaWcbJV1Vu+yAm33YBdR8aRyvD1g+pLUeGr0WhgNuePB3v++edxzTXXQKPRlLQwonM5OhAE%0AACzvYPhWO1EQcMWqFiTTOU68orqjXcgXv/7663j++efx9NNPz3g7l8sMrVbd4ezxcHP+Uin1tXxl%0Ae9/Uv3cd8QIA4lkJ7x3zz/h1NquxpHVUWrXXP53Tnx+fvHopfr+9H7uOenHTtcvK/ni0MLyWpVN0%0A+L711lt44okn8NRTT8Fmm/kHEgioew9Xj8cGrzeidBk1oRzXMhJNAgBykoRRfwwumwGZdBaZdLak%0Aj6MmNqtx6vuuNac/P4wisLjFhj1HvDjW54fDoi/pY/F3u3R4LedvpjcrRXU7RyIRbN68GT//+c/h%0AdDqLLoxoPnyhJHKSjGaXSelSqISuXN0CSZax8+CY0qUQVUxR4fvSSy8hEAjg7rvvxu23347bb78d%0Aw8PDpa6N6AxjEwkAQHODWeFKqJQuu6AZGlHA2wdGlC6FqGKK6na+9dZbceutt5a6FqIZFY6ga25g%0Ay7eW2M16rO5uwL5eP4a8UbR7uH6bah8XSlJVkCQZ3mACTqseRv2C5gmSCq1f3QIA+MuBUYUrIaoM%0Ahi9VBX8oiWxOZpdzjVq3vBFWkw5v7Rvmml+qCwxfqgqjhS5nTraqSTqtBteta0csmcXbbP1SHWD4%0AUlUYCxTGe9nyrVU3XNwOrUbAH94dgCTLSpdDVFYMX1I9SZIxHkjAYdHDZOB4b61yWA24/IJmjE3E%0A8X7vzBuoEFU7hi+p3kS4MN7LLudat/EjiwAAr707oHAlROXF8CXVOzXeyy7nWtfZbMPKLhcO9Qdw%0Acoy7KVHtYviS6g378uHb4maAeEmiAAAUe0lEQVT41oOPXZZv/b76Dlu/VLsYvqRqiVQW44E43HYD%0Ax3vrxOolbrQ1WrDj4CiGfDGlyyEqC4Yvqdqh/gAkGWjjrkd1QxQEfOHapZBl4Pk3jyldDlFZMHxJ%0A1fYfz896bW+0KFwJVdJFy9w4f5ET+3r9ONQfULocopJjPx6plizL2H/cD71ORKOzNs+2rUfb9g7N%0A6XZLOxw4MhDEL35/EJ9c3wVBEHDd2vYyV0dUGWz5kmoN+2KYCKfQ5rZAFASly6EKa3QY0d1qw0Q4%0AhRMjYaXLISophi+p1v7jEwCAdg+7nOvVuuUeiKKA3Ud9yGQlpcshKhmGL6lWYby3jeO9dctq1mHV%0AYhfiySx2HR5XuhyikmH4kiolUlkcHQiiq8XGJUZ1bs2yRrhsBvQMhrC3x6d0OUQlwfAlVTrcH0BO%0AknHhErfSpZDCNKKADWtaIYoCfvXyIYRjaaVLIlowhi+pUqHLeQ3DlwC4bAZcfF4jwvEMfvXyYcg8%0A9YiqHMOXVCcnSdh91AubWYfuNpvS5ZBKrOxyYUWnE3uP+fDSjn6lyyFaEIYvqc7hk0GE4xl8ZEUT%0ANCKfopQnCAK+/qlVcNkM+M0fj2PHwVGlSyIqGl/ZSHV2HhwDAFy+slnhSkhtXDYDvn3zRTAZNHj6%0A94dw5CR3v6LqxPAlVclkJew+4oXLZsCyDofS5ZAKdTRZ8c3PXghZBn62dT+GvFGlSyKaN4YvqcoH%0AJyYQT2Vx6Yom7mpF01q1uAFf/fgKxJJZ/Oi5Pegb5Q5YVF0YvqQqOw9NdjlfwC5nmtmGNa244xMr%0AEEtksPm5PeyCpqrC8CXVSKVz2NPjRZPThMUtnOVMs7vmojb87/+xCpmshEf/cx834aCqwfAl1djX%0A60M6I+GyC5ogsMuZ5uiylc3Y9Pk1EAD89Dfv4/fb+7gOmFSP4UuqUZjlfBlnOdM8rVnqxr1fvhjO%0AyWVIP3/xA6QyOaXLIpoWN80lVQhEUni/1492jwUdHqvS5ZBKzXYW8I0f6cC2PUN459A4egZD+Our%0AuqGBzHOASXXY8iVVeHPPEHKSjBsu6VC6FKpiJoMWH71sEc5b5EAgksJ/vn4U/aMRpcsiOgvDlxSX%0AzuSwbc8QLEYt1q9qUbocqnIaUcQVq1qwYU0LZBn4495hPPfaUWRzPA+Y1IPhS4rbcXAM0UQG165t%0Ah0GnUbocqhFL2hy4+YblcFj1eP29Qfzg2d3wBRNKl0UEgOFLCpNlGa/vGoAoCPirizkuR6XVYDfi%0Ar6/owvpVLTgxEsb/+dW72HuMy5FIeQxfUtTh/gAGvTF8ZIUHDXaj0uVQDdJpRfyvm1bijk+sQDor%0A4SfPv49fv3mM3dCkKIYvKeq1XYMAgI2XLlK4EqplgiDgmova8N3bL0Gzy4SXd57Ej/9jDwKRlNKl%0AUZ1i+JJiTo5FsO+YD0va7FjaxkMUqPw6m234hzsuxUdWNKFnMIT7n34HB074lS6L6hDX+ZIiZFnG%0Af7zeAxnAZzZ0K10O1bgPrw9e2eWEKAC7Do/j0S37sGapG2uWuc95mAfXCM9strXXM6nna8uWLyni%0AvSNeHBkIYu2yRqxe4la6HKozgiBgRZcLH7+iE1aTDu/3+vH6u4NIpLJKl0Z1guFLFZfO5LDlv49B%0AIwq49a+WKV0O1bFGhwmfvLILHU1WjE7E8bu/9GFgnOcDU/kxfKniXnnnJPzhJDZeugjNDWaly6E6%0AZ9BpcP26NnxkhQfprIQ3dw9hxwejyGQ5G5rKh2O+VFG+YAIv7eiH3azDp65crHQ5RADy3dAXLG5A%0Aq9uCt/YN4+hACMO+OC6/oEnp0qhGseVLFZPK5PCzF/YjnZFw8/XLYDLwvR+pi8tmwCfXd+GCxS7E%0Akhm88d4QHn9hPybCSaVLoxrDVz+qCFmW8auXD+PkWBTXXNSKK1dzD2dSJ41GxEdWNGFpux07PhjD%0Ae0e8eL/Xj2vWtOHjl3fC7eBmMLRwDF+qiFfeOYmdB8ewtN2O/7nxfAjnWNJBpCYumxEfv7wTOo0G%0AL/7lBN7YPYhte4ewflULNqxpxbIOxzmXJlGeJMkIxdKYCCcRTWSQTOeQTOeQy0nQ6zTQaUWEomks%0AarJicYsNLpuhrl4XGL4loNQ6t2Ift9Jr63YdHsfz23rhtOpx52cvhE7L0Q6qDoIgYMOaVlyxqhk7%0AD47h99v78ef9I/jz/hE02A24bGUzVi1uwNJ2O4z6M19OlXhdUHLNbSYrwRtMYHQijrGJOCbCKeQk%0AecavOXIyOPVvh0WPVd0NWLe8Eau73TDoa/uQFYYvlY0sy3hpRz9+88fjMOg0uPNzF8JpNShdFtG8%0AaTUirrqwFetXteBg/wR2HhzD7qNevLLzJF7ZeRKiIKCrxYrOZhtaG8xocVsQiadhMelqtnUcT2Zw%0AYiSCPUe9GJ2IwxdKQp7MWkEAnFYD3A4jGmwG2C16mAwaGHRaaDUC0lkJmWwOy9ud6B+LoG8kgmPD%0AIbx9YBRvHxiFTitizVI3rlzdgguXuKHV1N4bdoYvlUUmm8OvXj6M7R+MocFuwLc+vwadzTalyyJa%0AEFEUsLrbjdXdbnzlYzkc6g/gyEAQRweC6BuJ4MRI5Izba0QBdos+/8esg82sh82ig92sh1GvqVg3%0AqyTLyGYlpDPSmQdKnPbww74YBCH/PQqCgFwuf/tMVkIkkUbmiBcnR8IYC8TRPxqBL3RqEpogAG67%0AEc0NZrQ0mNHkMs3Yw6XXaQDosHqJe2qTHUmW0T8awZ4eL947cuqPzazD5Rc046rVrehsttZM1zTD%0AdxaSJGMsEMeQN4bxYALeYAK+YAKRRAaxRBbxVAbpTP7JLAj5g7yNeg0Mek3+b53mtI+1MBnyf+t1%0AInQaEZIkQxRPPZnSmRyiiczZf+IZhOJphKNphOJpJJJZBKIpZLISZFmGAAEQAK1GgEGXfzyTXgvb%0A5C+83aKD02qYfNKXjyzL2HV4HFv/dByjE3EsabNj0+cuhIMtXqoxOq0Ga5Y2Ys3SRgD5N5xjgQRG%0A/XGM+GPYe8yHcCyNUCx9zgMcdBoRNsvk76dZB7tFP/W7KsvynENGkmQEoylMRFI4MRJGPJlFLJFB%0ALJlFLJlBPJlFMp2b9X5e/HPfnL93q0mHVd0N6Gq2IZXJwuMyQa9d2GuLKAjobrWju9WOz169BP1j%0AEby9fxQ7Do7h9V2DeH3XINo9Fly5qgWXrGhCk9O0oMdTmiDL8syd8tN4+OGHsW/fPgiCgPvuuw9r%0A1qyZ9rZeb2Taz6mBx2OD1xtBIpXFkDeGgfEITo5HMTAexaA3OhWupzPoNbAYtTAbdEiks4AMADKy%0AORmpTA6pdA5zvbCCAEDGvG5vNmghCAJ0WhGikP9aWQayOQmpdA7paTYIMBu1WNbuwKImK9o9Fizy%0AWNHcYF5wt04qncOBE368+u4Ajg2GIAoCrlvXhluuX7bgwF/IOFa1slmNiES5vGWhSnEdFzr+Kssy%0AEqkswrEMwvE0IvE0IvEMwrH83+caFzUZtHDZDLAYtbAYddDrTv1+5iQ5H7DJ/BvzYCQNaZqXcY0o%0AwGLUwmTQQqfTQK8VodWIEATg1Jfk/9HmtkCS8/VKsgyNKEKvE6HX5l/rutqd0EKGx2mC06qfenNQ%0A7nHmbE7C/uN+vH1gFPuO+ZDN5evtarbhkvM9U28CTm/EqIXHM31vX1Et33feeQf9/f3YsmULent7%0Acd9992HLli1FF1hpiVQWI/44hnxRDPti8IZT6B8OwR8+892pRhTQ1mjBoiYrOjxWtDSY4XEa0egw%0AnTEZ4FxPPkmWkc5ISKWzSGZySKZyk7P9skikcshkc8hkJViMOuRkGQLyPUA6nQZWk+7sP2YdHGY9%0A7FY9rJPjSDM96SUp/wsfiWcQiefffQejKQQiabzf68f7vadOctFqBLS6LejwWNHqNqNx8ntssBlg%0AMmjP6h7L5iT4w0mMT77L/6BvAof6A1M7Al22sgmfvXoJd68iQn7Sltmog9moQ4v7zN8JWc4Hafgc%0AgRyOpTHij2G65pFBr4HVqMWSNjsa7AY02I2YCCdhNmphMelgMWph0M29a3u2ICw0UipNqxGxbrkH%0A65Z7EE1ksPuoF7uOjONQXwD9YxFs/dNxWIxanN/pwpI2Ozqb82PvdrO+4rXOR1Hhu337dtx4440A%0AgKVLlyIUCiEajcJqtZa0uOlEExmEovmZdFN/chJyUj7w4ql8l0t86k/+HWIgkoI/nEQsefbm6Q6r%0AHhcsdqG90YrOZisWNVnR1mgpukUoCgKMk13PMx2WV66Zx6Io5H8BTWf/wl9yngeD3hgGvVEMTrbu%0A8y3+c+9pKwiAUa+ZvM7yOd+pt3ssWLusER+/agksWvW9AyVSI0E49Xvaetr5IoXXBUmWkUxlkcpI%0AKGSoKAowG7TnfG2q9V4iq0mHay5qwzUXtSGWzGB/rx8H+wM41DeB3Ue92H3UO3Vbi1GLRocJjU4j%0AXFYDrOZ8Q8Zs0EKnzS91mvqjEfO3r2BXdlHh6/P5sGrVqqmPGxoa4PV6KxK+qXQO3/m/fzlnV/Bs%0ADDoNGuwGdLfa0dxgRnujBW2NFly0ohmJWP0cqm0z67GyS4+VXa6p/5MkGd5gAmOB/KxFXyiJYCSF%0AeCqLeCqLVDoHjShAq8k/WRtsBjS5TPC4TFja5oBn8kmr1LtjolokTrWala5EfSxGHa5Y1YIrVrVA%0AlmX4Q0n0j0XQPxbFwFgE48EEhv0x9I/N/fVo0+cvxLrlnjJWfUpRY75///d/j2uvvXaq9fvFL34R%0ADz/8MLq7eS4rERHRbIrqU21qaoLP55v6eHx8HB5PZd4tEBERVbuiwveqq67Cq6++CgD44IMP0NTU%0AVLHxXiIiompX1JjvxRdfjFWrVuG2226DIAi4//77S10XERFRzSp6nS8REREVp/Y2zCQiIlI5hi8R%0AEVGF1eXezplMBvfeey+Gh4eh0Wjwgx/8AIsWLTrjNi+99BKefvppiKKI9evX49vf/rZC1arXXK5j%0AKBTC3/7t38JiseAnP/mJQpWq10zbtL799tt49NFHodFocM011+DOO+9UsFL1m+laplIp/MM//AN6%0AenqwdetWBatUv5mu444dO/Doo49CFEV0d3fjoYcegiiyDVcUuQ5t3bpVfuCBB2RZluW33npLvuuu%0Au874fDwel6+//no5EonIkiTJX/jCF+Senh4lSlW12a6jLMvyXXfdJT/++OPypk2bKl2e6u3cuVP+%0Am7/5G1mWZfnYsWPyLbfccsbnP/GJT8jDw8NyLpeTv/jFL/I5OIPZruX3v/99+Ze//KX82c9+Vony%0AqsZs13Hjxo3yyMiILMuyvGnTJnnbtm0Vr7FW1OVblu3bt2Pjxo0AgCuvvBK7d+8+4/Mmkwkvvvgi%0ArNb88VVOpxPBYPBcd1XXZruOAPCP//iPuOSSSypdWlWYbptWABgYGIDD4UBraytEUcS1116L7du3%0AK1muqs10LQHg29/+9tTnaXqzXcetW7eipaUFQH5nw0AgoEidtaAuw9fn86GhoQEAIIoiBEFAOp0+%0A4zaFdctHjhzB0NAQLrrooorXqXbzuY50Np/PB5fr1BafhW1aAcDr9U5d2w9/js4207UE+Dycq7le%0Ax/HxcfzlL3/BtddeW/Eaa0XNj/n++te/xq9//esz/m/fvn1nfCxPs9qqr68P3/nOd/BP//RP0Ol0%0AZauxGizkOtLc8PqVDq9laZzrOvr9fnzjG9/A/ffff0ZQ0/zUfPjefPPNuPnmm8/4v3vvvRderxcr%0AVqxAJpOBLMvQ6888fmp0dBR33nknNm/ejJUrV1ayZFUq9jrS9GbapvXDnxsbG0NTU1PFa6wW3PK2%0ANGa7jtFoFF//+tdx9913Y8OGDUqUWDPqstv5qquuwiuvvAIAePPNN3H55ZefdZvvfve7eOCBB844%0AvYnONJfrSNObaZvWjo4ORKNRDA4OIpvN4s0338RVV12lZLmqxi1vS2O26/jDH/4QX/3qV3HNNdco%0AVWLNqMsdrnK5HL73ve+hr68Per0eP/zhD9Ha2op/+7d/w6WXXgqn04nPfOYzZ0yxv+OOO3DDDTco%0AWLX6zHYd16xZgzvuuAPhcBhjY2NYvnw5vvnNb2L9+vVKl64ajzzyCHbt2jW1TevBgwdhs9mwceNG%0AvPvuu3jkkUcAAB/96Efxta99TeFq1W2ma/mtb30Lo6Oj6OnpwerVq3HLLbfgU5/6lNIlq9J013HD%0Ahg249NJLsW7duqnb3nTTTbj11lsVrLZ61WX4EhERKakuu52JiIiUxPAlIiKqMIYvERFRhTF8iYiI%0AKozhS0REVGEMXyIiogpj+BIREVVYzW8vSVRLdu7ciSeeeAItLS3Yv38/LrroIpx//vl47bXXEAwG%0A8eSTT6Kvrw+PP/44ZFmGVqvFgw8+iEWLFuG1117DU089Bb1ej1wuh82bN6OjowO333471q9fjz17%0A9qCvrw+bNm3Cpz/9aaW/VaKaxpYvUZV5//33cc899+A3v/kNfve738Fut+OZZ57BqlWr8Nvf/hb3%0A338/fvrTn+LZZ5/Fl7/8ZWzevBkAEA6H8c///M945plncO211+Lf//3fp+4zHo/jySefxEMPPYSn%0AnnpKqW+NqG6w5UtUZZYuXQqn0wkAcDqdU9v9NTc3I5vNwuv1YtOmTQDyW4AKggAAaGxsxD333ANZ%0AluH1es/YJvCyyy4DALS1tSEUClXy2yGqSwxfoiqj0Wim/fjQoUNoa2vDM888c8ZtMpkM7r77brzw%0AwgtYvHgxnn32WRw4cGDq81rtqZcC7jhLVH7sdiaqIYsXL0YgEMDRo0cBAO+++y62bNmCWCwGURTR%0A3t6OVCqFN954A+l0WuFqieoXW75ENcRoNOLHP/4xvvvd78JgMAAAvv/978PpdOKmm27CF77wBbS1%0AteFrX/sa/u7v/g4vv/yywhUT1SeeakRERFRh7HYmIiKqMIYvERFRhTF8iYiIKozhS0REVGEMXyIi%0Aogpj+BIREVUYw5eIiKjCGL5EREQV9v8BvLIApin56dkAAAAASUVORK5CYII=%0A)

In [0]:
from statsmodels.stats.diagnostic import kstest_normal

result=[]
for i in z.columns:
  k,p=kstest_normal(z[i])
  result.append(p>=0.05)

print('Number of latent variables passed KS Normal Test: ', np.sum(result))

Number of latent variables passed KS Normal Test:  200

### 7.2 Low Dimensions Representation

We compare our VAE model, as a dimension reduction tool, with PCA and t-SNE. PCA is a commonly used dimension reduction method. It tries to find orthogonal dimensions and to represent samples under new space. The key idea is to maximize the variance of samples in each dimension and eliminate the linear correlation between dimensions. Hence, PCA would have a good performance if the collinearity problem is serious in origin features. PCA could also be regarded as a new space following multiple Gaussian distribution. And it would result in poor performance if the Gaussian assumption is violated. Hence, we consider t-SNE method, which could reduce dimensions in a non-linear way. Since the original input is too large for calculation on our computers, we consider only reduce dimensions on the 200 latent variables in our training samples. 

#### 7.2.1 PCA for 200 Latent Variables

The plot displays the explained variance ration for each dimension in range [1, 50]. The first 2 components explain only 9.5% of total variance. And first 50 components explain 89.2% of total variance. Hence the reduction using PCA is not a good choice. In some extends, the result also indicate good performance of latent variables under VAE, since they are good features with little correlation.

In [0]:
from sklearn.decomposition import PCA

n=int(z.shape[0]*1)
samples=z.sample(n)
samples=samples*100

pca = PCA(n_components=50)
pca.fit(samples)
print(pca.explained_variance_ratio_) 
print('total explained variance: ', np.sum(pca.explained_variance_ratio_))

fig, ax = plt.subplots()
ax.plot(pca.explained_variance_ratio_)


ax.set(xlabel='Dimensions', ylabel='Explained Variance Ratio',
       title='Performance of PCA on Latent Z')
plt.show()

Total explained variance for first 20th variables:  0.8925

![替代文字](...%0A)

#### 7.2.2 t-SNE for 200 Latent Variables

From the result of t-SNE, we can see that there are indeed some clusters, at least 2 clusters, in two dimensions. We multiply 100 to values in 200 latent variables in order to avoid floating point inaccuracy. There is a small group in the top of the plot, and a big group in the rest of the plot. By reducing dimensions via t-SNE, 200 latent variables are projected in a 2 dimensions space with distinctive clusters. 

In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [0]:
#Sub sample using 2%
n=int(z.shape[0]*0.02)
samples=z.sample(n)
samples.shape

#Did t-SNE
z_embedded=TSNE(n_components=2,perplexity=30.0).fit_transform(samples*100)
z_embedded=pd.DataFrame(z_embedded)


#Plot
fig, ax = plt.subplots()
ax.scatter(z_embedded[0],z_embedded[1],alpha=0.5)


ax.set(xlabel='Dimension 1', ylabel='Dimension 2',
       title='Result of t-SNE')

plt.show()

![替代文字](...%0A)

#### 7.2.3 VAE with 2 Latent Variables

t-SNE has a disadvantage that the locations of samples in new space is a random variable following some distribution. Thus, locations might variate every time. This problem makes the popular t-SNE be only a good visualization tool. We add a layer of 2 nodes in our original model with 200 latent variables, so as to reduce the dimension to 2. We multiply 100 to values in 200 latent variables in order to avoid floating point inaccuracy. The result seems good. There are clearly 3 clusters in the plot. Comparing with result from t-SNE, clusters in VAE are denser. It makes more sense to consider each crowd as a cluster and further investigate the detail of samples in every cluster. Besides, locations of samples under new space could be determinant if we use expectation to predict the location. They could also be random variables if we resample from distributions of latent variables. Thus, we highly recommend to use VAE to reduce dimension. It still has high level performance even we make a trade-off between generation and prediction. 

In [0]:
n=int(z.shape[0]*0.02)
samples=z.sample(n)
samples.shape


fig, ax = plt.subplots()
ax.scatter(samples[0]*100,samples[1]*100,alpha=0.5)

ax.set(xlabel='Z1', ylabel='Z2',title='Plot of Users in 2 Latent Variable')
plt.show()

![替代文字](...%0A)

### 7.3 Clustering Using 200 Latent Variables 

We think it would be interesting to cluster using latent variables in VAE. Because it makes sense to group 2 people together if they all give high rates on Scientific Fiction movies, which in some aspects indicate their common interests. Since our VAE model with 200 latent variables perform the best on recommendation test, we use these 200 variables to do the K-means clustering, one of the commonly used method. We use all samples in the training data for learning. In order to converge fast, K-means ++ method are used to select initial center points. It also saves computational sources, otherwise we need to repeat the learning process with different initial points to avoid local minima. 


The choice of k, the number of clusters, is crucial and a challenge in the K-means method. In our experiment, the principal method to select k, Elbow method, is employed. The main idea of Elbow method is to select the last k value, under which there is a significant improvement for the performance of clustering. We use Inertia as the metric of performance of clustering. Hence, the last k with significant decreasing of Inertia should be selected. The result of clustering, however, is not very good. As could be seen, there is not an appropriate 'elbow' point in the range of [2, 20]. The model might suffer from the curse of dimensionality, since we have only about 100k samples but 200 dimensions. The violation of the assumption, that all clusters are normally distributed, could be a reason cause the poor performance of K-means clustering. We highly recommend to use VAE with fewer latent variables or other clustering method, like DBSCAN to explore the usefulness of VAE for clustering, even under the trade-off between generation and prediction.


In [0]:
from sklearn.cluster import KMeans

minRes=2**31

for i in range(2,20):
  kmeans = KMeans(n_clusters=i, random_state=1).fit(z)
  elbow.append(kmeans.inertia_)
  if kmeans.inertia_<minRes:
    minRes=kmeans.inertia_
    label=kmeans.labels_.copy()

In [0]:
fig, ax = plt.subplots()
ax.plot(range(2,20),elbow)


ax.set(xlabel='Numbers of Clusters', ylabel='Inertia',
       title='Elbow Plot')

plt.show()

![替代文字](...%0A)

## 8. Conclusion

In this project, we strongly argue that the results of Liang's paper [1] using an inappropriate process to train and validate model. The result of repeat the experiment supports our statement. There is a more applicable way to deal with data. That is, arranging data following the timestamp. Although our result is lower than that of the paper according to the metrics Recall, our way to deal with data considering timestamp makes more sense in the real-world application. 

We further introduce our Predictive VAE, which make more regularization on the generative power of latent variable on input matrix. The novel training method hides last 20% samples’ value in time order as input, while computing the loss function, Entropy, uses entire data. This train trick delivers better performance than the original one. 

We also try to exploit the input matrix by build a deeper neural network regarding the famous ladder VAE. The performance of ladder VAE is not as good as our Predictive VAE. In some extends, it makes sense. Because ladder VAE is a way to more extensively learn the pattern of input, which improve the power of generation of VAE model. However, we have claimed that by reduce the power of generation and reinforce the ability of prediction, models would perform better.

Moreover, our Predictive VAE is still a powerful tool for dimension reduction and visualization. It combine the advantage of PCA and t-SNE and could have a more distinctive projection into low dimensions.
.

## 9. Reference


[1] Dawen Liang, Rahul G. Krishnan, Matthew D. Hoffman and Tony Jebara. 2018. Variational Autoencoders for Collaborative Filtering. arXiv preprint arXiv:1802.05814v1 (2018).

[2] Diederik P. Kingma and Max Welling. 2013. Auto-encoding variational bayes. arXiv preprint arXiv:1312.6114 (2013).

[3] Dawen Liang, Jaan Altosaar, Laurent Charlin, and David M. Blei. 2016. Factor- ization meets the item embedding: Regularizing matrix factorization with item co-occurrence. In Proceedings of the 10th ACM conference on recommender systems. 59–66.

[4] David M. Blei, Alp Kucukelbir, and Jon D. McAuliffe. 2017. Variational Inference: A Review for Statisticians. J. Amer. Statist. Assoc. 112, 518 (2017), 859–877.

[5] Michael I. Jordan, Zoubin Ghahramani, Tommi S. Jaakkola, and Lawrence K. Saul. 1999. An introduction to variational methods for graphical models. Machine learning 37, 2 (1999), 183–233.

[6] Samuel R. Bowman, Luke Vilnis, Oriol Vinyals, Andrew M. Dai, Rafal Jozefowicz,
and Samy Bengio. 2015. Generating sentences from a continuous space. arXiv
preprint arXiv:1511.06349 (2015).

[7] Irina Higgins, Loic Matthey, Arka Pal, Christopher Burgess, Xavier Glorot,
Matthew Botvinick, Shakir Mohamed, and Alexander Lerchner. 2017. β-VAE: Learning Basic Visual Concepts with a Constrained Variational Framework. In 5th International Conference on Learning Representations.

[8] Diederik Kingma and Jimmy Ba. 2014. Adam: A method for stochastic optimiza- tion. arXiv preprint arXiv:1412.6980 (2014).

[9] Hu, Yifan, Yehuda Koren, and Chris Volinsky. "Collaborative filtering for implicit feedback datasets." Data Mining, 2008. ICDM'08. Eighth IEEE International Conference on. Ieee, 2008. 

[10] Casper Kaae Sønderby. 2016. Ladder Variational Autoencoders. arXiv preprint arXiv:1312.6114 (2016).